# t_zi_radical

This notebook will merge 214 Kangxi radical from wikipedia and elemental zi from Zinets app,
load them into t_zi_radical table as a reference data 

- http://en.wikipedia.org/wiki/List_of_Kangxi_radicals  - Kangxi radicals
- https://www.xiaoma.info/bushou.php?ext=1  - 小马词典

python code is generated by Claude: https://claude.ai/chat/c513eb77-e271-495f-ac35-e05d1657732a


```
conda activate zinet

cd C:\Users\p2p2l\projects\wgong\zistory\zinets\dev\notebook
```

In [1]:
excel_path = "elemental_zi_v2.xlsx"
db_path = "elemental_zi_v2.sqlite3"

In [2]:
import pandas as pd
import sqlite3
import re
from pathlib import Path

def convert_to_snake_case(name):
    """Convert a string to snake_case."""
    # Replace spaces and special characters with underscores
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    # Convert to lowercase and replace multiple underscores with single
    x = re.sub('[-\s]+', '_', s2.lower().strip())
    r = [i.strip() for i in x.split("_") if i.strip()]
    return "_".join(r)

def excel_to_sqlite(excel_path, db_path, table_prefix='sh_', line_sep = 80 * "*"):
    """
    Load Excel sheets into SQLite with snake_case column names.
    
    Args:
        excel_path (str): Path to the Excel file
        db_path (str): Path where the SQLite database should be created/loaded
        table_prefix (str): Prefix to use for table names (default: 'sheet_')
    
    Returns:
        list: Names of created tables
    """
    try:
        # Read all sheets from Excel file
        excel_file = pd.ExcelFile(excel_path)
        created_tables = []
        
        # Connect to SQLite
        conn = sqlite3.connect(db_path)
        
        for sheet_name in excel_file.sheet_names:
            # Read the sheet
            df = pd.read_excel(excel_file, sheet_name=sheet_name).fillna("")
            
            # Convert column names to snake_case
            df.columns = [convert_to_snake_case(col) for col in df.columns]
            
            # Create valid table name
            table_name = f"{table_prefix}{convert_to_snake_case(sheet_name)}"
            
            # Load dataframe into SQLite
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            
            created_tables.append(table_name)
            print(f"\n{line_sep}\n Created table: {table_name}\n{line_sep}\n")
            
            # Print schema
            print(f"\nColumns:")
            schema_result = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
            for row in schema_result:
                print(f"\t{row[1]} \t [ Type: {row[2]} ]")
            
            # Print sample data
            print("\nFirst 5 rows:")
            sample_data = conn.execute(f"SELECT * FROM {table_name} LIMIT 5").fetchall()
            if sample_data:
                # Get column names
                columns = [row[1] for row in schema_result]
                # Print column headers
                print(" | ".join(columns))
                print("-" * (len(" | ".join(columns))))
                # Print data rows
                for row in sample_data:
                    print(" | ".join(str(val) for val in row))
            else:
                print("No data in table")
            
            print("\n" + "="*50 + "\n")
        
        conn.close()
        return created_tables
        
    except Exception as e:
        print(f"Error: {str(e)}")
        raise

def main():
  
    try:
        created_tables = excel_to_sqlite(excel_path, db_path)
        print(f"Successfully created tables: {created_tables}")
    except Exception as e:
        print(f"Failed to process Excel file: {str(e)}")

In [3]:
def normalize_unicode_string(text: str) -> str:
    """
    Normalizes a string by removing non-Unicode characters, control characters,
    and special characters like parentheses and punctuation.
    
    Args:
        text (str): Input string to normalize
        
    Returns:
        str: Normalized string containing only valid Unicode characters without special characters
    """
    # Filter out non-printable characters and normalize Unicode
    normalized = ''.join(char for char in text if char.isprintable())
    
    # Remove specific Unicode control characters like \xa0
    normalized = ''.join(char for char in normalized if ord(char) < 0xFFFF)
    
    # Remove special characters, keeping only letters and numbers
    normalized = ' '.join(char for char in normalized if not char in '、(){}[]<>,.;:!?@#$%^&*_+-=~/\\|\'\"')
    
    return " ".join([c.strip() for c in normalized.split() if c.strip()])

# Example usage:
text = '手\xa0(扌龵)'
result = normalize_unicode_string(text)
print(result)  # Output: '手扌龵'

手 扌 龵


In [4]:
def process_zi_items(data, name="meaning"):
    result = {}
    for item in data:
        zi = item['zi']
        item = item[name]
        
        # Skip empty meanings
        if not item:
            continue
        
        if isinstance(item, int):
            item = str(item)
            
        if isinstance(item, str) and item.endswith(".0"):
            item = item.replace(".0", "")
            
        # If zi exists in result, add to its set; otherwise create new set
        if zi in result:
            result[zi].add(item)
        else:
            result[zi] = {item}
            
    return result

# Example usage:
data = [{'zi': '么', 'meaning': 'small'}, {'zi': '么', 'meaning': 'short thread'}, {'zi': '以', 'meaning': ''}]
result = process_zi_items(data, name="meaning")
print(result)  # Output: {'么': {'short thread'}}

{'么': {'short thread', 'small'}}


In [5]:
def create_df_from_list(data_list):
    # Create DataFrame directly from the list of dictionaries
    df = pd.DataFrame(data_list)
    
    # Optional: Convert is_zi to boolean type
#     df['is_zi'] = df['is_zi'].astype(bool)
    
    return df

# Example usage:
data_is_zi = [{'zi': '么', 'is_zi': 1},
              {'zi': '以', 'is_zi': 0},
              {'zi': '⺷', 'is_zi': 0},
              {'zi': '丈', 'is_zi': 1},
              {'zi': '戶', 'is_zi': 0},
              {'zi': '耒', 'is_zi': 1},
              {'zi': '彳', 'is_zi': 0},
              {'zi': '东', 'is_zi': 0},
              {'zi': '疋', 'is_zi': 0},
              {'zi': '攵', 'is_zi': 0},
              {'zi': '舟', 'is_zi': 1}]

df = create_df_from_list(data_is_zi)
df

,zi,is_zi
0,么,1
1,以,0
2,⺷,0
3,丈,1
4,戶,0
5,耒,1
6,彳,0
7,东,0
8,疋,0
9,攵,0


In [6]:
if __name__ == "__main__":
    main()   # saved output to readme-prepare_t_zi_radical.md


********************************************************************************
 Created table: sh_elemental_zi_v2
********************************************************************************


Columns:
	zi 	 [ Type: TEXT ]
	is_radical_kangxi 	 [ Type: TEXT ]
	is_zi 	 [ Type: TEXT ]
	zi_meaning 	 [ Type: TEXT ]
	meaning 	 [ Type: TEXT ]
	pinyin 	 [ Type: TEXT ]
	zi_related 	 [ Type: TEXT ]
	zi_traditional 	 [ Type: TEXT ]
	zinets_analyzed 	 [ Type: TEXT ]
	n_strokes 	 [ Type: TEXT ]
	n_frequency 	 [ Type: TEXT ]
	is_active 	 [ Type: TEXT ]

First 5 rows:
zi | is_radical_kangxi | is_zi | zi_meaning | meaning | pinyin | zi_related | zi_traditional | zinets_analyzed | n_strokes | n_frequency | is_active
----------------------------------------------------------------------------------------------------------------------------------------------------
乚 | y |  | 一 |  |  |  |  |  | 1.0 |  | 
一 | y | y |  | one |  |  |  |  | 1.0 |  | 
乙 | y | y | 一 |  |  |  |  |  | 1.0 |  | 
丶 | y |  | 

### remove newlines from selected columns

In [7]:
sql_1 = """
UPDATE sh_kangxi_radical_214
SET 
    zi_radical = REPLACE(REPLACE(zi_radical, CHAR(13)||CHAR(10), ' '), CHAR(10), ' '),
    colloquial_term = REPLACE(REPLACE(colloquial_term, CHAR(13)||CHAR(10), ' '), CHAR(10), ' ')
WHERE 1=1 
-- and id_kangxi = 18
and (zi_radical  LIKE '%' || CHAR(10) || '%'
   OR zi_radical  LIKE '%' || CHAR(13) || CHAR(10) || '%'
   OR colloquial_term  LIKE '%' || CHAR(10) || '%'
   OR colloquial_term  LIKE '%' || CHAR(13) || CHAR(10) || '%');
"""


In [8]:
sql_2 = """
UPDATE sh_kangxi_radical_214_wikipedia
SET 
    radical_forms = REPLACE(REPLACE(radical_forms, CHAR(13)||CHAR(10), ' '), CHAR(10), ' '),
    colloquial_term = REPLACE(REPLACE(colloquial_term, CHAR(13)||CHAR(10), ' '), CHAR(10), ' ')
WHERE 1=1 
-- and id_kangxi = 18
and (radical_forms  LIKE '%' || CHAR(10) || '%'
   OR radical_forms  LIKE '%' || CHAR(13) || CHAR(10) || '%'
   OR colloquial_term  LIKE '%' || CHAR(10) || '%'
   OR colloquial_term  LIKE '%' || CHAR(13) || CHAR(10) || '%');
"""


In [9]:
with sqlite3.connect(db_path) as conn: 
    conn.execute(sql_1)
    conn.execute(sql_2)

## collect all radicals

In [10]:
sql_3 = """
with z as (

    select zi 
    from sh_elemental_zi_v2
    union
    select zi 
    from sh_w_radical_1
    union
    select zi 
    from sh_w_radical_2
    union
    select zi_radical as zi 
    from sh_kangxi_radical_214
    union
    select zi_radical as zi 
    from sh_xiaoma

)
select z.zi, length(z.zi) as len from z where length(z.zi) > 0
"""
with sqlite3.connect(db_path) as conn: 
    df = pd.read_sql(sql_3, conn)

In [11]:
df 

,zi,len
0,⺀,1
1,⺁,1
2,⺌,1
3,⺍,1
4,⺧,1
...,...,...
561,龟,1
562,龠,1
563,龴,1
564,龺,1


In [12]:
data = df.to_dict("records")

In [13]:
len(data), data

(566,
 [{'zi': '⺀', 'len': 1},
  {'zi': '⺁', 'len': 1},
  {'zi': '⺌', 'len': 1},
  {'zi': '⺍', 'len': 1},
  {'zi': '⺧', 'len': 1},
  {'zi': '⺩', 'len': 1},
  {'zi': '⺮', 'len': 1},
  {'zi': '⺶', 'len': 1},
  {'zi': '⺷', 'len': 1},
  {'zi': '⻏', 'len': 1},
  {'zi': '㐬', 'len': 1},
  {'zi': '㔾', 'len': 1},
  {'zi': '㕻', 'len': 1},
  {'zi': '一', 'len': 1},
  {'zi': '丁', 'len': 1},
  {'zi': '七', 'len': 1},
  {'zi': '万', 'len': 1},
  {'zi': '丈', 'len': 1},
  {'zi': '三', 'len': 1},
  {'zi': '上', 'len': 1},
  {'zi': '下', 'len': 1},
  {'zi': '不', 'len': 1},
  {'zi': '与', 'len': 1},
  {'zi': '丑', 'len': 1},
  {'zi': '专', 'len': 1},
  {'zi': '且', 'len': 1},
  {'zi': '丘', 'len': 1},
  {'zi': '丙', 'len': 1},
  {'zi': '业', 'len': 1},
  {'zi': '东', 'len': 1},
  {'zi': '丨', 'len': 1},
  {'zi': '个', 'len': 1},
  {'zi': '丬', 'len': 1},
  {'zi': '中', 'len': 1},
  {'zi': '丰', 'len': 1},
  {'zi': '丶', 'len': 1},
  {'zi': '丷', 'len': 1},
  {'zi': '丸', 'len': 1},
  {'zi': '为', 'len': 1},
  {'zi': '丿', 'len'

## prepare an unique set of radicals 

In [14]:
alias = set()
radicals = set()
for d in data: 
    z = d.get('zi').strip()
    l = d.get('len')
    if l > 1:
        n = normalize_unicode_string(z)
        alias.add(n)
        for c in n:
            radicals.add(c)
    elif len(z) and z not in [' ']:
        radicals.add(z)
    

In [15]:
len(alias), alias 

(97,
 {'丿 乀 ⺄',
  '丿 乀 乁',
  '乙 乚 乛',
  '人 亻',
  '八 丷',
  '刀 刂',
  '刀 刂 ⺈',
  '卤 鹵',
  '卩 㔾',
  '小 ⺌ ⺍',
  '尢 尣',
  '巛 川',
  '川 巛 巜',
  '幺 么',
  '彐 彑',
  '心 忄 ⺗',
  '心 忄 㣺',
  '戶 户 戸',
  '手 扌 龵',
  '攴 攵',
  '无 旡',
  '歹 歺',
  '毋 母',
  '毋 母 ⺟',
  '水 氵 氺',
  '火 灬',
  '爪 爫',
  '爿 丬',
  '牛 牜',
  '牛 牜 ⺧',
  '犬 犭',
  '玉 王 ⺩',
  '玉 王 玊',
  '用 甩',
  '疋 ⺪',
  '目 ⺫',
  '示 礻',
  '禺',
  '竹 ⺮',
  '糸 糹',
  '糸 糹 纟',
  '网 ⺲ 罓 ⺳',
  '网 罓 罒 ⺳',
  '羊 ⺶',
  '羊 ⺶ ⺷',
  '老 耂',
  '聿 ⺺ ⺻',
  '聿 肀',
  '肉 ⺼',
  '肉 月',
  '艸 ⺿',
  '艸 艹',
  '衣 ⻂',
  '衣 衤',
  '襾 西 覀',
  '西 襾 覀',
  '見 见',
  '角 ⻇',
  '言 訁',
  '言 訁 讠',
  '貝 贝',
  '走 赱',
  '足 ⻊',
  '車 车',
  '辵 ⻌ ⻍ ⻎',
  '辵 辶',
  '邑 ⻏',
  '邑 阝',
  '金 釒',
  '金 釒 钅',
  '長 镸',
  '長 长',
  '門 门',
  '阜 ⻖',
  '阜 阝',
  '靑 青',
  '青 靑',
  '面 靣',
  '韋 韦',
  '頁 页',
  '風 风',
  '飛 飞',
  '食 飠',
  '食 飠 饣',
  '馬 马',
  '高 髙',
  '魚 鱼',
  '鳥 鸟',
  '麥 麦',
  '黃 黄',
  '黽 黾',
  '鼓 鼔',
  '齊 斉',
  '齊 齐',
  '齒 齿',
  '龍 龙',
  '龜 龟'})

In [16]:
alias = {'丿 乀 ⺄',
  '丿 乀 乁',
  '乙 乚 乛',
  '人 亻',
  '八 丷',
  '刀 刂',
  '刀 刂 ⺈',
  '卤 鹵',
  '卩 㔾',
  '小 ⺌ ⺍',
  '尢 尣',
  '巛 川',
  '川 巛 巜',
  '幺 么',
  '彐 彑',
  '心 忄 ⺗',
  '心 忄 㣺',
  '戶 户 戸',
  '手 扌 龵',
  '攴 攵',
  '无 旡',
  '歹 歺',
  '毋 母',
  '毋 母 ⺟',
  '水 氵 氺',
  '火 灬',
  '爪 爫',
  '爿 丬',
  '牛 牜',
  '牛 牜 ⺧',
  '犬 犭',
  '玉 王 ⺩',
  '玉 王 玊',
  '用 甩',
  '疋 ⺪',
  '目 ⺫',
  '示 礻',
  '禺',
  '竹 ⺮',
  '糸 糹',
  '糸 糹 纟',
  '网 ⺲ 罓 ⺳',
  '网 罓 罒 ⺳',
  '羊 ⺶',
  '羊 ⺶ ⺷',
  '老 耂',
  '聿 ⺺ ⺻',
  '聿 肀',
  '肉 ⺼',
  '肉 月',
  '艸 ⺿',
  '艸 艹',
  '衣 ⻂',
  '衣 衤',
  '襾 西 覀',
  '西 襾 覀',
  '見 见',
  '角 ⻇',
  '言 訁',
  '言 訁 讠',
  '貝 贝',
  '走 赱',
  '足 ⻊',
  '車 车',
  '辵 ⻌ ⻍ ⻎',
  '辵 辶',
  '邑 ⻏',
  '邑 阝',
  '金 釒',
  '金 釒 钅',
  '長 镸',
  '長 长',
  '門 门',
  '阜 ⻖',
  '阜 阝',
  '靑 青',
  '青 靑',
  '面 靣',
  '韋 韦',
  '頁 页',
  '風 风',
  '飛 飞',
  '食 飠',
  '食 飠 饣',
  '馬 马',
  '高 髙',
  '魚 鱼',
  '鳥 鸟',
  '麥 麦',
  '黃 黄',
  '黽 黾',
  '鼓 鼔',
  '齊 斉',
  '齊 齐',
  '齒 齿',
  '龍 龙',
  '龜 龟'}

In [17]:
variant_str = """
一： 丶 丨 亅 丿 乀 ⺄ 乁 乙 乚 乛 
二： 亠 冫 丷 刂 龴
人： 亻
八： 丷
刀： 刂 ⺈
卤： 鹵
小： ⺌ ⺍
犬 犭
川： 巛 巜
幺： 么
爪： 爫
心： 忄 ⺗ 㣺
目 ⺫
户： 戸 戶 
手： 扌 龵
牛 牜 ⺧
无： 旡
歹： 歺
母： 毋 ⺟
水： 氵 冫 氺 川
火： 灬
玉 王 ⺩ 玊
片 爿 丬
用 甩
示 礻
竹 ⺮
草 艹 ⺿ 艸 
糸 糹 纟
网 ⺲ 罒 ⺳ 罓 
羊 ⺶ ⺷
老 耂
聿 ⺺  肀 ⺻
肉 ⺼
月 ⺼ 
衣 ⻂ 衤
西 襾 覀
见 見 
角 ⻇
言 訁 讠
贝 貝
走 ⻌ ⻍ ⻎ 辶 辵 赱
足 ⻊ 疋 ⺪
车 車
邑 ⻏ 阝
阜 ⻖ 阝
金 釒 钅
长 長 镸
门 門 
青 靑
面 靣
韦 韋 
页 頁 
风 風 
飞 飛 
食 飠 饣
马 馬 
高 髙
鱼 魚 
鸟 鳥 
麦 麥 
黄 黃 
黾 黽 
鼓 鼔
齐 齊 斉
齿 齒 
龙 龍 
龟 龜
互： 彐 彑
又： 攴 攵
卩 㔾
"""

In [18]:
variant = [i.strip().replace(":", " ").replace("：", " ") for i in variant_str.split("\n") if i.strip()]

In [19]:
data_variant = []
for i in variant:
    x = [j.strip() for j in i.split() if j.strip()]
    if len(x) < 2:
        print(x) 
        continue 
    zi = x[0] 
    var = " ".join(x[1:])
    data_variant.append({"zi": zi, "variant": var})
    
data_variant

[{'zi': '一', 'variant': '丶 丨 亅 丿 乀 ⺄ 乁 乙 乚 乛'},
 {'zi': '二', 'variant': '亠 冫 丷 刂 龴'},
 {'zi': '人', 'variant': '亻'},
 {'zi': '八', 'variant': '丷'},
 {'zi': '刀', 'variant': '刂 ⺈'},
 {'zi': '卤', 'variant': '鹵'},
 {'zi': '小', 'variant': '⺌ ⺍'},
 {'zi': '犬', 'variant': '犭'},
 {'zi': '川', 'variant': '巛 巜'},
 {'zi': '幺', 'variant': '么'},
 {'zi': '爪', 'variant': '爫'},
 {'zi': '心', 'variant': '忄 ⺗ 㣺'},
 {'zi': '目', 'variant': '⺫'},
 {'zi': '户', 'variant': '戸 戶'},
 {'zi': '手', 'variant': '扌 龵'},
 {'zi': '牛', 'variant': '牜 ⺧'},
 {'zi': '无', 'variant': '旡'},
 {'zi': '歹', 'variant': '歺'},
 {'zi': '母', 'variant': '毋 ⺟'},
 {'zi': '水', 'variant': '氵 冫 氺 川'},
 {'zi': '火', 'variant': '灬'},
 {'zi': '玉', 'variant': '王 ⺩ 玊'},
 {'zi': '片', 'variant': '爿 丬'},
 {'zi': '用', 'variant': '甩'},
 {'zi': '示', 'variant': '礻'},
 {'zi': '竹', 'variant': '⺮'},
 {'zi': '草', 'variant': '艹 ⺿ 艸'},
 {'zi': '糸', 'variant': '糹 纟'},
 {'zi': '网', 'variant': '⺲ 罒 ⺳ 罓'},
 {'zi': '羊', 'variant': '⺶ ⺷'},
 {'zi': '老', 'variant': '耂'},


In [20]:
len(radicals), radicals

(492,
 {' ',
  '⺀',
  '⺁',
  '⺄',
  '⺈',
  '⺌',
  '⺍',
  '⺗',
  '⺟',
  '⺧',
  '⺩',
  '⺪',
  '⺫',
  '⺮',
  '⺲',
  '⺳',
  '⺶',
  '⺷',
  '⺺',
  '⺻',
  '⺼',
  '⺿',
  '⻂',
  '⻇',
  '⻊',
  '⻌',
  '⻍',
  '⻎',
  '⻏',
  '⻖',
  '㐬',
  '㔾',
  '㕻',
  '㣺',
  '一',
  '丁',
  '七',
  '万',
  '丈',
  '三',
  '上',
  '下',
  '不',
  '与',
  '丑',
  '专',
  '且',
  '丘',
  '丙',
  '业',
  '东',
  '丨',
  '个',
  '丬',
  '中',
  '丰',
  '丶',
  '丷',
  '丸',
  '为',
  '丿',
  '乀',
  '乁',
  '乂',
  '乃',
  '久',
  '么',
  '义',
  '之',
  '乌',
  '乍',
  '乎',
  '乐',
  '乙',
  '乚',
  '乛',
  '九',
  '乞',
  '也',
  '习',
  '乡',
  '亅',
  '了',
  '二',
  '亍',
  '于',
  '亏',
  '云',
  '五',
  '井',
  '亠',
  '亡',
  '亢',
  '交',
  '人',
  '亻',
  '今',
  '介',
  '仓',
  '令',
  '以',
  '佥',
  '儿',
  '兀',
  '元',
  '兄',
  '入',
  '八',
  '公',
  '六',
  '兰',
  '共',
  '其',
  '冂',
  '内',
  '冈',
  '冓',
  '冖',
  '冫',
  '冬',
  '几',
  '凵',
  '凶',
  '出',
  '击',
  '刀',
  '刁',
  '刂',
  '刃',
  '分',
  '刍',
  '力',
  '办',
  '勹',
  '勺',
  '勾',
  '勿',
  '匀',
  '包',
  '匕',
  '北',
  '匚',


## prepare attribute by merging from various source tables

In [21]:
conn = sqlite3.connect(db_path)

In [22]:
tables = ['sh_elemental_zi_v2', 'sh_kangxi_radical_214', 
          'sh_xiaoma', 'sh_w_radical_1', 'sh_w_radical_2', 
          'sh_kangxi_radical_214_wikipedia']

###     attrib = is_zi

In [23]:
data_is_zi = [] 
n_max = 1000
n = 0
DEBUG = False 
for r in radicals:
    if r == ' ': continue

    sql_4 = f"""
    with m as ( 
        select zi, case when lower(is_zi) = 'y' then 1 else 0 end as is_zi 
        from sh_elemental_zi_v2 
        where trim(zi) = '{r}'
        
        union 
        
        select zi, is_zi 
        from sh_w_radical_1
        where trim(zi) = '{r}'
    )
    select * from m
    """
    df_2 = pd.read_sql(sql_4, conn)
    dd = df_2.to_dict("records")
    # pick one is_zi = 1 
    if len(dd) < 1:
#         print(f"[ERROR] {dd}")
        continue 
    d0 = dd[0]
    for i in range(1,len(dd)):
        d = dd[i]
        if d.get('is_zi'):
            d0.update(d)
            break 
    if DEBUG: 
        print(dd, "::", d0)
    data_is_zi.append(d0)
    
    n += 1 
    if n > n_max: break 
        
len(data_is_zi), data_is_zi

(419,
 [{'zi': '豕', 'is_zi': 0},
  {'zi': '夊', 'is_zi': 0},
  {'zi': '⺷', 'is_zi': 0},
  {'zi': '刀', 'is_zi': 1},
  {'zi': '乐', 'is_zi': 0},
  {'zi': '支', 'is_zi': 1},
  {'zi': '斗', 'is_zi': 1},
  {'zi': '黑', 'is_zi': 1},
  {'zi': '欠', 'is_zi': 1},
  {'zi': '⺍', 'is_zi': 0},
  {'zi': '本', 'is_zi': 0},
  {'zi': '⺮', 'is_zi': 0},
  {'zi': '鼻', 'is_zi': 1},
  {'zi': '开', 'is_zi': 0},
  {'zi': '内', 'is_zi': 0},
  {'zi': '方', 'is_zi': 1},
  {'zi': '邑', 'is_zi': 0},
  {'zi': '彐', 'is_zi': 0},
  {'zi': '九', 'is_zi': 1},
  {'zi': '肉', 'is_zi': 1},
  {'zi': '竹', 'is_zi': 1},
  {'zi': '尧', 'is_zi': 0},
  {'zi': '香', 'is_zi': 1},
  {'zi': '几', 'is_zi': 1},
  {'zi': '川', 'is_zi': 1},
  {'zi': '尼', 'is_zi': 0},
  {'zi': '辛', 'is_zi': 1},
  {'zi': '之', 'is_zi': 1},
  {'zi': '冖', 'is_zi': 0},
  {'zi': '术', 'is_zi': 0},
  {'zi': '斥', 'is_zi': 0},
  {'zi': '厂', 'is_zi': 1},
  {'zi': '丬', 'is_zi': 0},
  {'zi': '黍', 'is_zi': 0},
  {'zi': '犬', 'is_zi': 1},
  {'zi': '釆', 'is_zi': 1},
  {'zi': '市', 'is_zi':

In [24]:
data_is_zi = [
  {'zi': '么', 'is_zi': 1},
  {'zi': '以', 'is_zi': 1},
  {'zi': '⺷', 'is_zi': 0},
  {'zi': '丈', 'is_zi': 1},
  {'zi': '戶', 'is_zi': 1},
  {'zi': '耒', 'is_zi': 1},
  {'zi': '彳', 'is_zi': 0},
  {'zi': '东', 'is_zi': 1},
  {'zi': '疋', 'is_zi': 0},
  {'zi': '攵', 'is_zi': 0},
  {'zi': '舟', 'is_zi': 1},
  {'zi': '乂', 'is_zi': 0},
  {'zi': '饣', 'is_zi': 0},
  {'zi': '虍', 'is_zi': 0},
  {'zi': '刍', 'is_zi': 0},
  {'zi': '兄', 'is_zi': 1},
  {'zi': '阜', 'is_zi': 1},
  {'zi': '⺀', 'is_zi': 0},
  {'zi': '亠', 'is_zi': 0},
  {'zi': '麻', 'is_zi': 1},
  {'zi': '赤', 'is_zi': 1},
  {'zi': '青', 'is_zi': 1},
  {'zi': '耳', 'is_zi': 1},
  {'zi': '㐬', 'is_zi': 0},
  {'zi': '禾', 'is_zi': 1},
  {'zi': '丙', 'is_zi': 1},
  {'zi': '己', 'is_zi': 1},
  {'zi': '丨', 'is_zi': 0},
  {'zi': '吉', 'is_zi': 1},
  {'zi': '辰', 'is_zi': 1},
  {'zi': '入', 'is_zi': 1},
  {'zi': '可', 'is_zi': 1},
  {'zi': '屯', 'is_zi': 1},
  {'zi': '巴', 'is_zi': 1},
  {'zi': '石', 'is_zi': 1},
  {'zi': '儿', 'is_zi': 1},
  {'zi': '皮', 'is_zi': 1},
  {'zi': '匸', 'is_zi': 0},
  {'zi': '雨', 'is_zi': 1},
  {'zi': '讠', 'is_zi': 0},
  {'zi': '十', 'is_zi': 1},
  {'zi': '亻', 'is_zi': 0},
  {'zi': '⺩', 'is_zi': 0},
  {'zi': '车', 'is_zi': 1},
  {'zi': '竹', 'is_zi': 1},
  {'zi': '疒', 'is_zi': 0},
  {'zi': '平', 'is_zi': 1},
  {'zi': '奉', 'is_zi': 1},
  {'zi': '牜', 'is_zi': 0},
  {'zi': '厶', 'is_zi': 0},
  {'zi': '其', 'is_zi': 1},
  {'zi': '欠', 'is_zi': 1},
  {'zi': '勹', 'is_zi': 0},
  {'zi': '虫', 'is_zi': 1},
  {'zi': '高', 'is_zi': 1},
  {'zi': '干', 'is_zi': 1},
  {'zi': '耂', 'is_zi': 0},
  {'zi': '癶', 'is_zi': 0},
  {'zi': '斤', 'is_zi': 1},
  {'zi': '比', 'is_zi': 1},
  {'zi': '乍', 'is_zi': 0},
  {'zi': '业', 'is_zi': 1},
  {'zi': '齿', 'is_zi': 1},
  {'zi': '羽', 'is_zi': 1},
  {'zi': '另', 'is_zi': 1},
  {'zi': '坙', 'is_zi': 0},
  {'zi': '㕻', 'is_zi': 1},
  {'zi': '片', 'is_zi': 1},
  {'zi': '甲', 'is_zi': 1},
  {'zi': '飞', 'is_zi': 1},
  {'zi': '自', 'is_zi': 1},
  {'zi': '才', 'is_zi': 1},
  {'zi': '矢', 'is_zi': 1},
  {'zi': '又', 'is_zi': 1},
  {'zi': '衤', 'is_zi': 0},
  {'zi': '行', 'is_zi': 1},
    
  {'zi': '𤴔', 'is_zi': 0},
  {'zi': '央', 'is_zi': 1},
  {'zi': '少', 'is_zi': 1},
  {'zi': '里', 'is_zi': 1},
  {'zi': '句', 'is_zi': 1},
  {'zi': '万', 'is_zi': 1},
  {'zi': '屰', 'is_zi': 0},
  {'zi': '巾', 'is_zi': 1},
  {'zi': '令', 'is_zi': 1},
  {'zi': '肉', 'is_zi': 1},
  {'zi': '乁', 'is_zi': 0},
  {'zi': '丷', 'is_zi': 0},
  {'zi': '小', 'is_zi': 1},
  {'zi': '丘', 'is_zi': 1},
  {'zi': '口', 'is_zi': 1},
  {'zi': '于', 'is_zi': 1},
  {'zi': '夊', 'is_zi': 0},
  {'zi': '未', 'is_zi': 1},
  {'zi': '仓', 'is_zi': 1},
  {'zi': '黾', 'is_zi': 0},
  {'zi': '犭', 'is_zi': 1},
  {'zi': '囟', 'is_zi': 0},
  {'zi': '并', 'is_zi': 1},
  {'zi': '氺', 'is_zi': 0},
  {'zi': '生', 'is_zi': 1},
  {'zi': '亡', 'is_zi': 1},
  {'zi': '北', 'is_zi': 1},
  {'zi': '糸', 'is_zi': 1},
  {'zi': '金', 'is_zi': 1},
  {'zi': '首', 'is_zi': 1},
  {'zi': '木', 'is_zi': 1},
  {'zi': '今', 'is_zi': 1},
  {'zi': '西', 'is_zi': 1},
  {'zi': '犬', 'is_zi': 1},
  {'zi': '爫', 'is_zi': 0},
  {'zi': '冈', 'is_zi': 1},
  {'zi': '⺧', 'is_zi': 0},
  {'zi': '布', 'is_zi': 1},
  {'zi': '色', 'is_zi': 1},
  {'zi': '电', 'is_zi': 1},
    
  {'zi': '矛', 'is_zi': 1},
  {'zi': '刁', 'is_zi': 1},
  {'zi': '米', 'is_zi': 1},
  {'zi': '舌', 'is_zi': 1},
  {'zi': '酉', 'is_zi': 0},
  {'zi': '由', 'is_zi': 1},
  {'zi': '冬', 'is_zi': 1},
  {'zi': '牙', 'is_zi': 1},
  {'zi': '亍', 'is_zi': 0},
  {'zi': '个', 'is_zi': 1},
  {'zi': '母', 'is_zi': 1},
  {'zi': '扌', 'is_zi': 0},
  {'zi': '孚', 'is_zi': 0},
  {'zi': '匕', 'is_zi': 1},
  {'zi': '术', 'is_zi': 1},
  {'zi': '斗', 'is_zi': 1},
  {'zi': '旦', 'is_zi': 1},
  {'zi': '习', 'is_zi': 1},
  {'zi': '尺', 'is_zi': 1},
  {'zi': '鬼', 'is_zi': 1},
  {'zi': '畐', 'is_zi': 0},
  {'zi': '勾', 'is_zi': 1},
  {'zi': '丰', 'is_zi': 1},
  {'zi': '网', 'is_zi': 1},
  {'zi': '云', 'is_zi': 1},
  {'zi': '尢', 'is_zi': 0},
  {'zi': '勿', 'is_zi': 1},
  {'zi': '匚', 'is_zi': 0},
  {'zi': '卩', 'is_zi': 0},
  {'zi': '马', 'is_zi': 1},
  {'zi': '子', 'is_zi': 1},
  {'zi': '辛', 'is_zi': 1},
  {'zi': '龙', 'is_zi': 1},
  {'zi': '无', 'is_zi': 1},
  {'zi': '黑', 'is_zi': 1},
  {'zi': '宁', 'is_zi': 1},
  {'zi': '田', 'is_zi': 1},
  {'zi': '音', 'is_zi': 1},
  {'zi': '庄', 'is_zi': 1},
  {'zi': '⺍', 'is_zi': 0},
  {'zi': '臼', 'is_zi': 1},
  {'zi': '乚', 'is_zi': 0},
  {'zi': '㔾', 'is_zi': 0},
  {'zi': '香', 'is_zi': 1},
  {'zi': '鱼', 'is_zi': 1},
  {'zi': '殳', 'is_zi': 0},
  {'zi': '戈', 'is_zi': 1},
  {'zi': '氵', 'is_zi': 0},
    
  {'zi': '它', 'is_zi': 1},
  {'zi': '鸟', 'is_zi': 1},
  {'zi': '釆', 'is_zi': 1},
  {'zi': '凵', 'is_zi': 0},
  {'zi': '走', 'is_zi': 1},
  {'zi': '刂', 'is_zi': 0},
  {'zi': '丶', 'is_zi': 0},
  {'zi': '不', 'is_zi': 1},
  {'zi': '击', 'is_zi': 1},
  {'zi': '交', 'is_zi': 1},
  {'zi': '乐', 'is_zi': 1},
  {'zi': '肙', 'is_zi': 0},
  {'zi': '隶', 'is_zi': 1},
  {'zi': '卜', 'is_zi': 1},
  {'zi': '久', 'is_zi': 1},
  {'zi': '方', 'is_zi': 1},
  {'zi': '瓜', 'is_zi': 1},
  {'zi': '彐', 'is_zi': 0},
  {'zi': '奂', 'is_zi': 0},
  {'zi': '二', 'is_zi': 1},
  {'zi': '非', 'is_zi': 1},
  {'zi': '几', 'is_zi': 1},
  {'zi': '皿', 'is_zi': 1},
  {'zi': '王', 'is_zi': 1},
  {'zi': '刃', 'is_zi': 1},
  {'zi': '牛', 'is_zi': 1},
    
    
  {'zi': '冫', 'is_zi': 0},
  {'zi': '风', 'is_zi': 1},
  {'zi': '中', 'is_zi': 1},
  {'zi': '之', 'is_zi': 1},
  {'zi': '⺌', 'is_zi': 0},
  {'zi': '爿', 'is_zi': 0},
  {'zi': '白', 'is_zi': 1},
  {'zi': '页', 'is_zi': 1},
  {'zi': '足', 'is_zi': 1},
  {'zi': '忄', 'is_zi': 0},
  {'zi': '乙', 'is_zi': 1},
  {'zi': '鼻', 'is_zi': 1},
  {'zi': '龟', 'is_zi': 1},
  {'zi': '柬', 'is_zi': 1},
  {'zi': '氏', 'is_zi': 1},
  {'zi': '贝', 'is_zi': 1},
  {'zi': '广', 'is_zi': 1},
  {'zi': '灬', 'is_zi': 0},
  {'zi': '日', 'is_zi': 1},
  {'zi': '八', 'is_zi': 1},
  {'zi': '廴', 'is_zi': 0},
  {'zi': '立', 'is_zi': 1},
  {'zi': '衣', 'is_zi': 1},
  {'zi': '为', 'is_zi': 1},
  {'zi': '四', 'is_zi': 1},
  {'zi': '工', 'is_zi': 1},
  {'zi': '九', 'is_zi': 1},
  {'zi': '大', 'is_zi': 1},
  {'zi': '友', 'is_zi': 1},
  {'zi': '一', 'is_zi': 1},
  {'zi': '⺶', 'is_zi': 0},
  {'zi': '凶', 'is_zi': 1},
  {'zi': '匀', 'is_zi': 1},
  {'zi': '宀', 'is_zi': 0},
  {'zi': '乀', 'is_zi': 0},
  {'zi': '止', 'is_zi': 1},
  {'zi': '乃', 'is_zi': 1},
  {'zi': '東', 'is_zi': 1},
  {'zi': '⺮', 'is_zi': 0},
  {'zi': '千', 'is_zi': 1},
  {'zi': '彡', 'is_zi': 0},
  {'zi': '六', 'is_zi': 1},
  {'zi': '黍', 'is_zi': 0},
  {'zi': '尚', 'is_zi': 1},
  {'zi': '禺 ', 'is_zi': 1},
  {'zi': '冂', 'is_zi': 0},
  {'zi': '向', 'is_zi': 1},
  {'zi': '各', 'is_zi': 1},
  {'zi': '⻏', 'is_zi': 0},
  {'zi': '办', 'is_zi': 1},
  {'zi': '支', 'is_zi': 1},
  {'zi': '与', 'is_zi': 1},
  {'zi': '义', 'is_zi': 1},
    
    
    
  {'zi': '寺', 'is_zi': 1},
  {'zi': '夂', 'is_zi': 0},
  {'zi': '且', 'is_zi': 1},
  {'zi': '瓦', 'is_zi': 1},
  {'zi': '下', 'is_zi': 1},
  {'zi': '幺', 'is_zi': 1},
  {'zi': '申', 'is_zi': 1},
  {'zi': '毋', 'is_zi': 1},
  {'zi': '勺', 'is_zi': 1},
  {'zi': '甘', 'is_zi': 1},
  {'zi': '亅', 'is_zi': 0},
  {'zi': '舛', 'is_zi': 1},
  {'zi': '至', 'is_zi': 1},
  {'zi': '公', 'is_zi': 1},
  {'zi': '夆', 'is_zi': 0},
  {'zi': '山', 'is_zi': 1},
  {'zi': '太', 'is_zi': 1},
  {'zi': '面', 'is_zi': 1},
  {'zi': '曲', 'is_zi': 1},
  {'zi': '七', 'is_zi': 1},
  {'zi': '卤', 'is_zi': 1},
  {'zi': '爪', 'is_zi': 1},
  {'zi': '聿', 'is_zi': 0},
  {'zi': '豸', 'is_zi': 0},
  {'zi': '廾', 'is_zi': 1},
  {'zi': '父', 'is_zi': 1},
  {'zi': '已', 'is_zi': 1},
  {'zi': '刀', 'is_zi': 1},
  {'zi': '韦', 'is_zi': 1},
  {'zi': '甫', 'is_zi': 1},
  {'zi': '及', 'is_zi': 1},
  {'zi': '见', 'is_zi': 1},
  {'zi': '市', 'is_zi': 1},
  {'zi': '吕', 'is_zi': 1},
  {'zi': '艹', 'is_zi': 0},
  {'zi': '尧', 'is_zi': 0},
  {'zi': '古', 'is_zi': 1},
  {'zi': '夕', 'is_zi': 1},
  {'zi': '阝', 'is_zi': 0},
  {'zi': '冖', 'is_zi': 0},
  {'zi': '乞', 'is_zi': 1},
  {'zi': '辶', 'is_zi': 0},
  {'zi': '丬', 'is_zi': 0},
  {'zi': '午', 'is_zi': 1},
  {'zi': '员', 'is_zi': 1},
  {'zi': '礻', 'is_zi': 0},
  {'zi': '水', 'is_zi': 1},
    
    
  {'zi': '言', 'is_zi': 1},
  {'zi': '心', 'is_zi': 1},
  {'zi': '也', 'is_zi': 1},
  {'zi': '爻', 'is_zi': 0},
  {'zi': '覀', 'is_zi': 0},
  {'zi': '韭', 'is_zi': 1},
  {'zi': '良', 'is_zi': 1},
  {'zi': '乌', 'is_zi': 1},
  {'zi': '丸', 'is_zi': 1},
  {'zi': '艮', 'is_zi': 0},
  {'zi': '区', 'is_zi': 1},
  {'zi': '丁', 'is_zi': 1},
  {'zi': '弋', 'is_zi': 1},
  {'zi': '必', 'is_zi': 1},
  {'zi': '卡', 'is_zi': 1},
  {'zi': '角', 'is_zi': 1},
  {'zi': '责', 'is_zi': 1},
  {'zi': '乎', 'is_zi': 1},
  {'zi': '尸', 'is_zi': 1},
  {'zi': '歹', 'is_zi': 1},
  {'zi': '五', 'is_zi': 1},
  {'zi': '出', 'is_zi': 1},
  {'zi': '谷', 'is_zi': 1},
  {'zi': '老', 'is_zi': 1},
  {'zi': '尤', 'is_zi': 1},
  {'zi': '豕', 'is_zi': 0},
  {'zi': '巨', 'is_zi': 1},
  {'zi': '血', 'is_zi': 1},
  {'zi': '户', 'is_zi': 1},
  {'zi': '麦', 'is_zi': 1},
  {'zi': '示', 'is_zi': 1},
  {'zi': '火', 'is_zi': 1},
  {'zi': '⺁', 'is_zi': 0},
  {'zi': '人', 'is_zi': 1},
  {'zi': '夫', 'is_zi': 1},
  {'zi': '巛', 'is_zi': 0},
  {'zi': '骨', 'is_zi': 1},
  {'zi': '气', 'is_zi': 1},
  {'zi': '寸', 'is_zi': 1},
  {'zi': '龺', 'is_zi': 0},
  {'zi': '士', 'is_zi': 1},
  {'zi': '呙', 'is_zi': 0},
  {'zi': '彑', 'is_zi': 0},
  {'zi': '式', 'is_zi': 1},
  {'zi': '专', 'is_zi': 1},
  {'zi': '月', 'is_zi': 1},
  {'zi': '玉', 'is_zi': 1},
  {'zi': '毛', 'is_zi': 1},
  {'zi': '用', 'is_zi': 1},
  {'zi': '井', 'is_zi': 1},
    
  {'zi': '罒', 'is_zi': 0},
  {'zi': '失', 'is_zi': 1},
  {'zi': '包', 'is_zi': 1},
  {'zi': '头', 'is_zi': 1},
  {'zi': '畏', 'is_zi': 1},
  {'zi': '匹', 'is_zi': 1},
  {'zi': '臣', 'is_zi': 1},
  {'zi': '元', 'is_zi': 1},
  {'zi': '曰', 'is_zi': 1},
  {'zi': '共', 'is_zi': 1},
  {'zi': '穴', 'is_zi': 1},
  {'zi': '咸', 'is_zi': 1},
  {'zi': '合', 'is_zi': 1},
  {'zi': '亏', 'is_zi': 1},
  {'zi': '亢', 'is_zi': 1},
  {'zi': '丑', 'is_zi': 1},
  {'zi': '占', 'is_zi': 1},
  {'zi': '本', 'is_zi': 1},
  {'zi': '攴', 'is_zi': 0},
  {'zi': '只', 'is_zi': 1},
  {'zi': '半', 'is_zi': 1},
  {'zi': '玄', 'is_zi': 1},
  {'zi': '罓', 'is_zi': 0},
  {'zi': '女', 'is_zi': 1},
  {'zi': '厂', 'is_zi': 1},
  {'zi': '佥', 'is_zi': 0},
  {'zi': '禸', 'is_zi': 0},
  {'zi': '丿', 'is_zi': 0},
  {'zi': '末', 'is_zi': 1},
  {'zi': '鼓', 'is_zi': 1},
  {'zi': '龴', 'is_zi': 0},
  {'zi': '斥', 'is_zi': 1},
  {'zi': '豆', 'is_zi': 1},
  {'zi': '升', 'is_zi': 1},
  {'zi': '天', 'is_zi': 1},
  {'zi': '去', 'is_zi': 1},
  {'zi': '上', 'is_zi': 1},
  {'zi': '内', 'is_zi': 1},
  {'zi': '食', 'is_zi': 1},
  {'zi': '隹', 'is_zi': 0},
  {'zi': '弓', 'is_zi': 1},
  {'zi': '正', 'is_zi': 1},
  {'zi': '身', 'is_zi': 1},
  {'zi': '乡', 'is_zi': 1},
  {'zi': '了', 'is_zi': 1},
  {'zi': '台', 'is_zi': 1},
  {'zi': '目', 'is_zi': 1},
  {'zi': '旡', 'is_zi': 0},

{'zi': '囗', 'is_zi': 1},
  {'zi': '齐', 'is_zi': 1},
  {'zi': '土', 'is_zi': 1},
  {'zi': '门', 'is_zi': 1},
  {'zi': '邑', 'is_zi': 1},
  {'zi': '戋', 'is_zi': 0},
  {'zi': '尼', 'is_zi': 1},
  {'zi': '力', 'is_zi': 1},
  {'zi': '开', 'is_zi': 1},
  {'zi': '叉', 'is_zi': 1},
  {'zi': '分', 'is_zi': 1},
  {'zi': '文', 'is_zi': 1},
  {'zi': '川', 'is_zi': 1},
  {'zi': '长', 'is_zi': 1},
  {'zi': '乛', 'is_zi': 0},
  {'zi': '钅', 'is_zi': 0},
  {'zi': '屮', 'is_zi': 0},
  {'zi': '幷', 'is_zi': 0},
  {'zi': '易', 'is_zi': 1},
  {'zi': '缶', 'is_zi': 1},
  {'zi': '羊', 'is_zi': 1},
  {'zi': '纟', 'is_zi': 0},
  {'zi': '革', 'is_zi': 1},
  {'zi': '而', 'is_zi': 1},
  {'zi': '三', 'is_zi': 1},
  {'zi': '黄', 'is_zi': 1},
  {'zi': '介', 'is_zi': 1},
  {'zi': '朩', 'is_zi': 0},
  {'zi': '兀', 'is_zi': 0},
  {'zi': '手', 'is_zi': 1},
  {'zi': '兰', 'is_zi': 1}]

###     attrib = id_kangxi

In [25]:
data_id_kangxi = [] 
n_max = 1000
n = 0
DEBUG = False # True # 
for r in radicals:
    if r == ' ': continue

    sql_5 = f"""
        select '{r}' as zi, id_kangxi
        from sh_kangxi_radical_214 
        where trim(zi_radical) = '{r}'
    """
    df_5 = pd.read_sql(sql_5, conn)
    dd = df_5.to_dict("records")
    if dd: 
        data_id_kangxi += dd
        if DEBUG: 
            print(dd)
    
    n += 1 
    if n > n_max: break 
        

len(data_id_kangxi), data_id_kangxi

(163,
 [{'zi': '豕', 'id_kangxi': 152},
  {'zi': '夊', 'id_kangxi': 35},
  {'zi': '車', 'id_kangxi': 159},
  {'zi': '風', 'id_kangxi': 182},
  {'zi': '支', 'id_kangxi': 65},
  {'zi': '鬥', 'id_kangxi': 191},
  {'zi': '斗', 'id_kangxi': 68},
  {'zi': '黑', 'id_kangxi': 203},
  {'zi': '欠', 'id_kangxi': 76},
  {'zi': '鼻', 'id_kangxi': 209},
  {'zi': '龠', 'id_kangxi': 214},
  {'zi': '方', 'id_kangxi': 70},
  {'zi': '香', 'id_kangxi': 186},
  {'zi': '几', 'id_kangxi': 16},
  {'zi': '辛', 'id_kangxi': 160},
  {'zi': '冖', 'id_kangxi': 14},
  {'zi': '厂', 'id_kangxi': 27},
  {'zi': '黍', 'id_kangxi': 202},
  {'zi': '釆', 'id_kangxi': 165},
  {'zi': '音', 'id_kangxi': 180},
  {'zi': '冂', 'id_kangxi': 13},
  {'zi': '彳', 'id_kangxi': 60},
  {'zi': '止', 'id_kangxi': 77},
  {'zi': '隶', 'id_kangxi': 171},
  {'zi': '生', 'id_kangxi': 100},
  {'zi': '厶', 'id_kangxi': 28},
  {'zi': '日', 'id_kangxi': 72},
  {'zi': '虫', 'id_kangxi': 142},
  {'zi': '儿', 'id_kangxi': 10},
  {'zi': '韋', 'id_kangxi': 178},
  {'zi': '田', 'id_

In [26]:
data_id_kangxi = [{'zi': '耒', 'id_kangxi': 127},
  {'zi': '彳', 'id_kangxi': 60},
  {'zi': '龠', 'id_kangxi': 214},
  {'zi': '魚', 'id_kangxi': 195},
  {'zi': '舟', 'id_kangxi': 137},
  {'zi': '虍', 'id_kangxi': 141},
  {'zi': '亠', 'id_kangxi': 8},
  {'zi': '麻', 'id_kangxi': 200},
  {'zi': '赤', 'id_kangxi': 155},
  {'zi': '耳', 'id_kangxi': 128},
  {'zi': '見', 'id_kangxi': 147},
  {'zi': '禾', 'id_kangxi': 115},
  {'zi': '己', 'id_kangxi': 49},
  {'zi': '丨', 'id_kangxi': 2},
  {'zi': '辰', 'id_kangxi': 161},
  {'zi': '入', 'id_kangxi': 11},
  {'zi': '石', 'id_kangxi': 112},
  {'zi': '儿', 'id_kangxi': 10},
  {'zi': '皮', 'id_kangxi': 107},
  {'zi': '韋', 'id_kangxi': 178},
  {'zi': '匸', 'id_kangxi': 23},
  {'zi': '雨', 'id_kangxi': 173},
  {'zi': '十', 'id_kangxi': 24},
  {'zi': '疒', 'id_kangxi': 104},
  {'zi': '黽', 'id_kangxi': 205},
  {'zi': '厶', 'id_kangxi': 28},
  {'zi': '欠', 'id_kangxi': 76},
  {'zi': '頁', 'id_kangxi': 181},
  {'zi': '勹', 'id_kangxi': 20},
  {'zi': '虫', 'id_kangxi': 142},
  {'zi': '干', 'id_kangxi': 51},
  {'zi': '癶', 'id_kangxi': 105},
  {'zi': '斤', 'id_kangxi': 69},
  {'zi': '鼎', 'id_kangxi': 206},
  {'zi': '比', 'id_kangxi': 81},
  {'zi': '羽', 'id_kangxi': 124},
  {'zi': '片', 'id_kangxi': 91},
  {'zi': '自', 'id_kangxi': 132},
  {'zi': '矢', 'id_kangxi': 111},
  {'zi': '又', 'id_kangxi': 29},
  {'zi': '行', 'id_kangxi': 144},
  {'zi': '里', 'id_kangxi': 166},
  {'zi': '巾', 'id_kangxi': 50},
  {'zi': '口', 'id_kangxi': 30},
  {'zi': '夊', 'id_kangxi': 35},
  {'zi': '鬯', 'id_kangxi': 192},
  {'zi': '生', 'id_kangxi': 100},
  {'zi': '首', 'id_kangxi': 185},
  {'zi': '木', 'id_kangxi': 75},
  {'zi': '麥', 'id_kangxi': 199},
  {'zi': '飛', 'id_kangxi': 183},
  {'zi': '色', 'id_kangxi': 139},
  {'zi': '矛', 'id_kangxi': 110},
  {'zi': '米', 'id_kangxi': 119},
  {'zi': '舌', 'id_kangxi': 135},
  {'zi': '鹵', 'id_kangxi': 197},
  {'zi': '酉', 'id_kangxi': 164},
  {'zi': '鬥', 'id_kangxi': 191},
  {'zi': '牙', 'id_kangxi': 92},
  {'zi': '匕', 'id_kangxi': 21},
  {'zi': '斗', 'id_kangxi': 68},
  {'zi': '鬼', 'id_kangxi': 194},
  {'zi': '車', 'id_kangxi': 159},
  {'zi': '風', 'id_kangxi': 182},
  {'zi': '匚', 'id_kangxi': 22},
  {'zi': '子', 'id_kangxi': 39},
  {'zi': '辛', 'id_kangxi': 160},
  {'zi': '黑', 'id_kangxi': 203},
  {'zi': '龍', 'id_kangxi': 212},
  {'zi': '田', 'id_kangxi': 102},
  {'zi': '音', 'id_kangxi': 180},
  {'zi': '臼', 'id_kangxi': 134},
  {'zi': '香', 'id_kangxi': 186},
  {'zi': '殳', 'id_kangxi': 79},
  {'zi': '戈', 'id_kangxi': 62},
  {'zi': '鬲', 'id_kangxi': 193},
  {'zi': '釆', 'id_kangxi': 165},
  {'zi': '凵', 'id_kangxi': 17},
  {'zi': '走', 'id_kangxi': 156},
  {'zi': '丶', 'id_kangxi': 3},
  {'zi': '隶', 'id_kangxi': 171},
  {'zi': '卜', 'id_kangxi': 25},
  {'zi': '方', 'id_kangxi': 70},
  {'zi': '瓜', 'id_kangxi': 97},
  {'zi': '二', 'id_kangxi': 7},
  {'zi': '非', 'id_kangxi': 175},
  {'zi': '几', 'id_kangxi': 16},
  {'zi': '皿', 'id_kangxi': 108},
  {'zi': '冫', 'id_kangxi': 15},
  {'zi': '白', 'id_kangxi': 106},
  {'zi': '鼻', 'id_kangxi': 209},
  {'zi': '氏', 'id_kangxi': 83},
  {'zi': '广', 'id_kangxi': 53},
  {'zi': '日', 'id_kangxi': 72},
  {'zi': '龜', 'id_kangxi': 213},
  {'zi': '黹', 'id_kangxi': 204},
  {'zi': '廴', 'id_kangxi': 54},
  {'zi': '立', 'id_kangxi': 117},
  {'zi': '黃', 'id_kangxi': 201},
  {'zi': '工', 'id_kangxi': 48},
  {'zi': '大', 'id_kangxi': 37},
  {'zi': '一', 'id_kangxi': 1},
  {'zi': '宀', 'id_kangxi': 40},
  {'zi': '止', 'id_kangxi': 77},
  {'zi': '彡', 'id_kangxi': 59},
  {'zi': '黍', 'id_kangxi': 202},
  {'zi': '冂', 'id_kangxi': 13},
  {'zi': '齒', 'id_kangxi': 211},
  {'zi': '支', 'id_kangxi': 65},
  {'zi': '夂', 'id_kangxi': 34},
  {'zi': '瓦', 'id_kangxi': 98},
  {'zi': '甘', 'id_kangxi': 99},
  {'zi': '亅', 'id_kangxi': 6},
  {'zi': '舛', 'id_kangxi': 136},
  {'zi': '至', 'id_kangxi': 133},
  {'zi': '髟', 'id_kangxi': 190},
  {'zi': '門', 'id_kangxi': 169},
  {'zi': '山', 'id_kangxi': 46},
  {'zi': '豸', 'id_kangxi': 153},
  {'zi': '廾', 'id_kangxi': 55},
  {'zi': '父', 'id_kangxi': 88},
  {'zi': '夕', 'id_kangxi': 36},
  {'zi': '冖', 'id_kangxi': 14},
  {'zi': '馬', 'id_kangxi': 187},
  {'zi': '爻', 'id_kangxi': 89},
  {'zi': '韭', 'id_kangxi': 179},
  {'zi': '艮', 'id_kangxi': 138},
  {'zi': '弋', 'id_kangxi': 56},
  {'zi': '尸', 'id_kangxi': 44},
  {'zi': '谷', 'id_kangxi': 150},
  {'zi': '豕', 'id_kangxi': 152},
  {'zi': '血', 'id_kangxi': 143},
  {'zi': '骨', 'id_kangxi': 188},
  {'zi': '气', 'id_kangxi': 84},
  {'zi': '寸', 'id_kangxi': 41},
  {'zi': '士', 'id_kangxi': 33},
  {'zi': '月', 'id_kangxi': 74},
  {'zi': '毛', 'id_kangxi': 82},
  {'zi': '用', 'id_kangxi': 101},
  {'zi': '臣', 'id_kangxi': 131},
  {'zi': '曰', 'id_kangxi': 73},
  {'zi': '鳥', 'id_kangxi': 196},
  {'zi': '穴', 'id_kangxi': 116},
  {'zi': '鼠', 'id_kangxi': 208},
  {'zi': '玄', 'id_kangxi': 95},
  {'zi': '女', 'id_kangxi': 38},
  {'zi': '厂', 'id_kangxi': 27},
  {'zi': '禸', 'id_kangxi': 114},
  {'zi': '鼓', 'id_kangxi': 207},
  {'zi': '豆', 'id_kangxi': 151},
  {'zi': '隹', 'id_kangxi': 172},
  {'zi': '弓', 'id_kangxi': 57},
  {'zi': '身', 'id_kangxi': 158},
  {'zi': '貝', 'id_kangxi': 154},
  {'zi': '囗', 'id_kangxi': 31},
  {'zi': '土', 'id_kangxi': 32},
  {'zi': '力', 'id_kangxi': 19},
  {'zi': '文', 'id_kangxi': 67},
  {'zi': '屮', 'id_kangxi': 45},
  {'zi': '缶', 'id_kangxi': 121},
  {'zi': '革', 'id_kangxi': 177},
  {'zi': '而', 'id_kangxi': 126},
  {'zi': '鹿', 'id_kangxi': 198}]

###     attrib = meaning

In [27]:
data_meaning = [] 
n_max = 1000  # 10  # 
n = 0
DEBUG = False # True #  
for r in radicals:
    if r == ' ': continue

    sql_5 = f"""
    with m as ( 
        select zi, lower(meaning) as meaning
        from sh_elemental_zi_v2 
        where trim(zi) = '{r}'
        
        union 
        
        select '{r}' as zi, lower(meaning) as meaning
        from sh_kangxi_radical_214
        where trim(zi_radical) like '%{r}%'
        union 
        
        select '{r}' as zi, lower(meaning) as meaning
        from sh_xiaoma
        where trim(zi_radical) like '%{r}%'
        union 
        
        select zi, lower(meaning) as meaning
        from sh_w_radical_2
        where trim(zi) = '{r}'        
    )
    select * from m
    """
    df_5 = pd.read_sql(sql_5, conn)
    dd = df_5.to_dict("records")
    if dd: 
        data_meaning += dd
        if DEBUG: 
            print(dd)
    
    n += 1 
    if n > n_max: break 

# len(data_meaning), data_meaning

result = process_zi_items(data_meaning, name="meaning")
data_meaning_2 = [{'zi': k, 'meaning':  " / ".join(v)} for k, v in result.items() if k != ' ']

len(data_meaning_2), data_meaning_2

(342,
 [{'zi': '豕', 'meaning': 'a pig, boar / pig'},
  {'zi': '夊', 'meaning': 'go slowly'},
  {'zi': '⺷', 'meaning': 'sheep'},
  {'zi': '刀', 'meaning': 'knife'},
  {'zi': '鼔', 'meaning': 'drum'},
  {'zi': '車', 'meaning': 'cart'},
  {'zi': '風', 'meaning': 'wind'},
  {'zi': '支', 'meaning': 'branch'},
  {'zi': '鬥', 'meaning': 'fight'},
  {'zi': '斗', 'meaning': 'dipper'},
  {'zi': '黑', 'meaning': 'black'},
  {'zi': '欠', 'meaning': 'owe / lack'},
  {'zi': '⺍', 'meaning': 'small'},
  {'zi': '⺮', 'meaning': 'bamboo'},
  {'zi': '襾', 'meaning': 'cover'},
  {'zi': '鼻', 'meaning': 'nose'},
  {'zi': '龠', 'meaning': 'flute'},
  {'zi': '⺲', 'meaning': 'net'},
  {'zi': '方', 'meaning': 'square'},
  {'zi': '邑', 'meaning': 'city (阝 right) / city'},
  {'zi': '彐', 'meaning': 'garment / snout'},
  {'zi': '九', 'meaning': 'nine'},
  {'zi': '肉', 'meaning': 'meat'},
  {'zi': '竹', 'meaning': 'bamboo'},
  {'zi': '香', 'meaning': 'fragrant'},
  {'zi': '几', 'meaning': 'table / jǐ, table'},
  {'zi': '川', 'meaning': 

In [28]:
data_meaning = [
    {'zi': '么', 'meaning': 'short thread'},
  {'zi': '⺷', 'meaning': 'sheep'},
  {'zi': '丈', 'meaning': 'meter'},
  {'zi': '戶', 'meaning': 'door'},
  {'zi': '耒', 'meaning': 'plow / plough'},
  {'zi': '彳', 'meaning': 'walk / step'},
  {'zi': '龠', 'meaning': 'flute'},
  {'zi': '魚', 'meaning': 'fish'},
  {'zi': '疋', 'meaning': 'foot / bolt of cloth'},
  {'zi': '攵', 'meaning': ' tap / rap'},
  {'zi': '舟', 'meaning': 'boat'},
  {'zi': '饣', 'meaning': 'eat'},
  {'zi': '虍', 'meaning': 'tiger'},
  {'zi': '辵', 'meaning': 'walk'},
  {'zi': '阜', 'meaning': 'dam (阝 left) / mound'},
  {'zi': '亠', 'meaning': 'lid'},
  {'zi': '麻', 'meaning': 'hemp'},
  {'zi': '赤', 'meaning': 'red'},
  {'zi': '⺻', 'meaning': 'brush'},
  {'zi': '青', 'meaning': 'green / blue'},
  {'zi': '耳', 'meaning': 'ear'},
  {'zi': '見', 'meaning': 'see'},
  {'zi': '禾', 'meaning': 'grain'},
  {'zi': '己', 'meaning': 'oneself'},
  {'zi': '丨', 'meaning': 'line'},
  {'zi': '辰', 'meaning': 'morning'},
  {'zi': '入', 'meaning': 'enter'},
  {'zi': '石', 'meaning': 'stone'},
  {'zi': '儿', 'meaning': 'son '},
  {'zi': '皮', 'meaning': 'skin'},
  {'zi': '韋', 'meaning': 'tanned leather / soft leather'},
  {'zi': '匸', 'meaning': 'hiding enclosure'},
  {'zi': '雨', 'meaning': 'rain'},
  {'zi': '讠', 'meaning': 'speech'},
    
  {'zi': '十', 'meaning': 'ten'},
  {'zi': '亻', 'meaning': 'man / person'},
  {'zi': '⺩', 'meaning': 'jade'},
  {'zi': '车', 'meaning': 'cart / vehicle'},
  {'zi': '竹', 'meaning': 'bamboo'},
  {'zi': '疒', 'meaning': 'sick / sickness'},
  {'zi': '黽', 'meaning': 'frog'},
  {'zi': '肀', 'meaning': 'brush'},
  {'zi': '牜', 'meaning': 'cow'},
  {'zi': '厶', 'meaning': ' private'},
  {'zi': '欠', 'meaning': 'owe / lack'},
  {'zi': '頁', 'meaning': 'leaf / page'},
  {'zi': '勹', 'meaning': 'wrap'},
  {'zi': '虫', 'meaning': 'small creature / insect'},
  {'zi': '高', 'meaning': 'tall'},
  {'zi': '干', 'meaning': 'dry'},
  {'zi': '耂', 'meaning': 'old'},
  {'zi': '癶', 'meaning': 'legs / footsteps'},
  {'zi': '斤', 'meaning': 'axe'},
  {'zi': '鼎', 'meaning': 'tripod'},
  {'zi': '⺪', 'meaning': 'foot / bolt of cloth'},
  {'zi': '比', 'meaning': 'compare'},
  {'zi': '乍', 'meaning': 'work'},
  {'zi': '齿', 'meaning': 'teeth'},
  {'zi': '羽', 'meaning': 'feather'},
  {'zi': '坙', 'meaning': 'stem'},
  {'zi': '片', 'meaning': 'slice'},
  {'zi': '飞', 'meaning': 'fly'},
  {'zi': '自', 'meaning': 'self'},
  {'zi': '才', 'meaning': 'talent'},
  {'zi': '矢', 'meaning': 'arrow'},
  {'zi': '又', 'meaning': 'hand / again'},
  {'zi': '衤', 'meaning': 'clothing'},
  {'zi': '行', 'meaning': 'walk enclosure / walk'},
  {'zi': '里', 'meaning': 'village / kilometer'},
  {'zi': '万', 'meaning': 'ten thousand'},
  {'zi': '巾', 'meaning': 'turban / cloth'},
  {'zi': '肉', 'meaning': 'meat / flesh'},
  {'zi': '乁', 'meaning': 'slash'},
  {'zi': '丷', 'meaning': 'eight'},
  {'zi': '小', 'meaning': 'small'},
  {'zi': '口', 'meaning': 'mouth / opening'},
  {'zi': '于', 'meaning': 'at'},
  {'zi': '夊', 'meaning': 'go slowly'},
  {'zi': '鬯', 'meaning': 'sacrificial wine'},
  {'zi': '黾', 'meaning': 'frog'},
  {'zi': '犭', 'meaning': 'dog'},
    
  {'zi': '斉', 'meaning': 'even'},
  {'zi': '氺', 'meaning': 'water'},
  {'zi': '生', 'meaning': 'life'},
  {'zi': '糸', 'meaning': 'silk'},
  {'zi': '金', 'meaning': 'metal / gold'},
  {'zi': '首', 'meaning': 'head'},
  {'zi': '木', 'meaning': 'tree / wood'},
  {'zi': '西', 'meaning': 'west / cover'},
  {'zi': '麥', 'meaning': 'wheat'},
  {'zi': '犬', 'meaning': 'dog'},
  {'zi': '飛', 'meaning': 'fly'},
  {'zi': '爫', 'meaning': 'claw'},
  {'zi': '⺺', 'meaning': 'brush'},
  {'zi': '⺧', 'meaning': 'cow'},
  {'zi': '色', 'meaning': 'color'},
  {'zi': '矛', 'meaning': 'spear'},
  {'zi': '米', 'meaning': 'rice'},
  {'zi': '舌', 'meaning': 'tongue'},
  {'zi': '鹵', 'meaning': 'salt'},
  {'zi': '酉', 'meaning': 'wine / wine vessel'},
  {'zi': '⺿', 'meaning': 'grass'},
  {'zi': '鬥', 'meaning': 'fight'},
  {'zi': '由', 'meaning': 'follow / reason'},
  {'zi': '牙', 'meaning': 'tooth / fang'},
  {'zi': '个', 'meaning': 'piece'},
  {'zi': '母', 'meaning': ' mother'},
  {'zi': '扌', 'meaning': 'hand / hold'},
  {'zi': '匕', 'meaning': 'dagger / spoon'},
  {'zi': '斗', 'meaning': 'dipper / fight'},
  {'zi': '訁', 'meaning': 'speech'},
  {'zi': '鬼', 'meaning': 'ghost'},
  {'zi': '車', 'meaning': 'cart'},
  {'zi': '⻖', 'meaning': 'mound'},
  {'zi': '風', 'meaning': 'wind'},
  {'zi': '歺', 'meaning': 'death / evil'},
  {'zi': '网', 'meaning': 'net'},
  {'zi': '尢', 'meaning': 'lame'},
  {'zi': '匚', 'meaning': 'right open box'},
  {'zi': '卩', 'meaning': 'seal (device) '},
  {'zi': '马', 'meaning': 'horse'},
  {'zi': '子', 'meaning': 'son / child'},
  {'zi': '辛', 'meaning': 'bitter'},
  {'zi': '⻌', 'meaning': 'walk'},
  {'zi': '龙', 'meaning': 'dragon'},
  {'zi': '无', 'meaning': 'not / without'},
  {'zi': '黑', 'meaning': 'black'},
  {'zi': '龍', 'meaning': 'dragon'},
    
    
  {'zi': '田', 'meaning': 'field'},
  {'zi': '音', 'meaning': 'sound'},
  {'zi': '長', 'meaning': 'long'},
  {'zi': '糹', 'meaning': 'silk'},
  {'zi': '⺍', 'meaning': 'small'},
  {'zi': '⺳', 'meaning': 'net'},
  {'zi': '臼', 'meaning': 'mortar'},
  {'zi': '乚', 'meaning': 'second'},
  {'zi': '㔾', 'meaning': 'seal (device) / seal'},
  {'zi': '尣', 'meaning': 'lame'},
  {'zi': '香', 'meaning': 'fragrant'},
  {'zi': '鱼', 'meaning': 'fish'},
  {'zi': '巜', 'meaning': 'river'},
  {'zi': '殳', 'meaning': 'weapon'},
  {'zi': '戈', 'meaning': 'halberd / spear'},
  {'zi': '氵', 'meaning': 'water'},
  {'zi': '鸟', 'meaning': 'bird'},
  {'zi': '鬲', 'meaning': 'cauldron'},
  {'zi': '釆', 'meaning': 'distinguish / discern'},
  {'zi': '凵', 'meaning': 'receptacle / open box'},
  {'zi': '走', 'meaning': 'run / walk'},
  {'zi': '刂', 'meaning': 'blade / knife'},
  {'zi': '丶', 'meaning': 'dot'},
  {'zi': '隶', 'meaning': 'slave'},
  {'zi': '卜', 'meaning': 'divination'},
  {'zi': '久', 'meaning': 'long'},
  {'zi': '方', 'meaning': 'square'},
  {'zi': '瓜', 'meaning': 'melon'},
  {'zi': '彐', 'meaning': 'garment / snout'},
  {'zi': '二', 'meaning': 'two'},
  {'zi': '非', 'meaning': 'negate / wrong'},
  {'zi': '几', 'meaning': ' table'},
  {'zi': '皿', 'meaning': 'dish'},
  {'zi': '王', 'meaning': 'jade'},
  {'zi': '牛', 'meaning': 'cow'},
  {'zi': '冫', 'meaning': 'ice / icy water'},
  {'zi': '风', 'meaning': 'wind'},
  {'zi': '⺼', 'meaning': 'meat'},
  {'zi': '⺌', 'meaning': 'small'},
  {'zi': '爿', 'meaning': 'split wood / piece of wood'},
  {'zi': '白', 'meaning': 'white'},
  {'zi': '页', 'meaning': 'page'},
  {'zi': '鼔', 'meaning': 'drum'},
  {'zi': '足', 'meaning': 'foot'},
  {'zi': '⻊', 'meaning': 'foot'},
  {'zi': '忄', 'meaning': 'heart / emotion'},
    
    
  {'zi': '乙', 'meaning': 'second'},
  {'zi': '鼻', 'meaning': 'nose'},
  {'zi': '龟', 'meaning': 'turtle'},
  {'zi': '氏', 'meaning': 'clan'},
  {'zi': '贝', 'meaning': 'shell'},
  {'zi': '广', 'meaning': 'broad / wide / shelter / dotted cliff'},
  {'zi': '灬', 'meaning': 'fire'},
  {'zi': '日', 'meaning': 'sun'},
  {'zi': '龜', 'meaning': 'turtle'},
  {'zi': '八', 'meaning': 'eight'},
  {'zi': '黹', 'meaning': 'embroidery / embroider'},
  {'zi': '廴', 'meaning': 'long stride'},
  {'zi': '立', 'meaning': 'stand'},
  {'zi': '衣', 'meaning': 'clothing / clothes'},
  {'zi': '黃', 'meaning': 'yellow'},
  {'zi': '工', 'meaning': 'work'},
  {'zi': '⺫', 'meaning': 'eye'},
  {'zi': '九', 'meaning': 'nine'},
  {'zi': '大', 'meaning': 'big'},
  {'zi': '一', 'meaning': 'one'},
  {'zi': '⺶', 'meaning': 'sheep'},
  {'zi': '髙', 'meaning': 'tall'},
  {'zi': '宀', 'meaning': 'roof'},
  {'zi': '⻂', 'meaning': 'clothes'},
  {'zi': '乀', 'meaning': 'slash'},
  {'zi': '止', 'meaning': 'stop'},
  {'zi': '⺮', 'meaning': 'bamboo'},
  {'zi': '千', 'meaning': 'thousand'},
  {'zi': '彡', 'meaning': 'hair / bristle'},
  {'zi': '黍', 'meaning': 'millet'},
  {'zi': '禺 ', 'meaning': 'join'},
  {'zi': '冂', 'meaning': 'wide / down box'},
  {'zi': '㣺', 'meaning': 'heart'},
  {'zi': '齒', 'meaning': 'tooth '},
  {'zi': '飠', 'meaning': 'eat'},
  {'zi': '⻏', 'meaning': 'city'},
  {'zi': '支', 'meaning': 'branch'},
  {'zi': '玊', 'meaning': 'jade'},
  {'zi': '龵', 'meaning': 'hand'},
  {'zi': '夂', 'meaning': 'go'},
  {'zi': '瓦', 'meaning': 'tile'},
  {'zi': '下', 'meaning': 'down'},
  {'zi': '幺', 'meaning': 'small / short thread'},
  {'zi': '戸', 'meaning': 'door / family'},
  {'zi': '毋', 'meaning': 'do not '},
  {'zi': '甘', 'meaning': 'sweet'},
    
    
  {'zi': '亅', 'meaning': 'hook'},
  {'zi': '靑', 'meaning': 'green / blue'},
  {'zi': '舛', 'meaning': 'oppose'},
  {'zi': '至', 'meaning': 'arrive / reach'},
  {'zi': '髟', 'meaning': 'hair'},
  {'zi': '門', 'meaning': 'door / gate'},
  {'zi': '山', 'meaning': 'mountain'},
  {'zi': '面', 'meaning': 'face'},
  {'zi': '七', 'meaning': 'seven'},
  {'zi': '卤', 'meaning': 'salt'},
  {'zi': '爪', 'meaning': 'claw'},
  {'zi': '聿', 'meaning': 'brush'},
  {'zi': '豸', 'meaning': 'fierce animal / badger'},
  {'zi': '廾', 'meaning': 'arch / two hands'},
  {'zi': '父', 'meaning': 'father'},
  {'zi': '镸', 'meaning': 'long'},
  {'zi': '已', 'meaning': 'already'},
  {'zi': '刀', 'meaning': 'knife'},
  {'zi': '⻎', 'meaning': 'walk'},
  {'zi': '韦', 'meaning': 'soft leather'},
  {'zi': '及', 'meaning': 'and'},
  {'zi': '见', 'meaning': 'see'},
  {'zi': '⺗', 'meaning': 'heart'},
  {'zi': '艹', 'meaning': 'grass'},
  {'zi': '夕', 'meaning': 'evening'},
  {'zi': '阝', 'meaning': 'mound, dam (阝 left) / city (阝 right)'},
  {'zi': '冖', 'meaning': 'cover / cloth cover'},
  {'zi': '辶', 'meaning': 'walk'},
  {'zi': '丬', 'meaning': 'split wood / piece of wood'},
  {'zi': '⺄', 'meaning': 'slash'},
  {'zi': '靣', 'meaning': 'face'},
  {'zi': '⻇', 'meaning': 'horn'},
  {'zi': '礻', 'meaning': 'spirit'},
  {'zi': '水', 'meaning': 'water'},
  {'zi': '馬', 'meaning': 'horse'},
  {'zi': '言', 'meaning': 'speech'},
  {'zi': '心', 'meaning': 'heart'},
  {'zi': '爻', 'meaning': 'line / trigrams'},
  {'zi': '覀', 'meaning': 'cover'},
  {'zi': '韭', 'meaning': 'leek'},
  {'zi': '齊', 'meaning': 'even / neat'},
  {'zi': '艮', 'meaning': 'stop and stare / stopping'},
  {'zi': '⻍', 'meaning': 'walk'},
  {'zi': '弋', 'meaning': 'shoot'},
  {'zi': '角', 'meaning': 'horn'},
  {'zi': '尸', 'meaning': 'corpse'},
  {'zi': '釒', 'meaning': 'gold'},
    
    
  {'zi': '歹', 'meaning': 'evil / death / bad'},
  {'zi': '谷', 'meaning': 'valley'},
  {'zi': '老', 'meaning': 'old'},
  {'zi': '豕', 'meaning': ' boar / pig'},
  {'zi': '⺈', 'meaning': 'knife'},
  {'zi': '血', 'meaning': 'blood'},
  {'zi': '户', 'meaning': 'door'},
  {'zi': '麦', 'meaning': 'wheat'},
  {'zi': '示', 'meaning': 'spirit / show'},
  {'zi': '火', 'meaning': 'fire'},
  {'zi': '人', 'meaning': 'man / person'},
  {'zi': '巛', 'meaning': 'river'},
  {'zi': '骨', 'meaning': 'bone'},
  {'zi': '气', 'meaning': 'steam'},
  {'zi': '寸', 'meaning': 'inch'},
  {'zi': '士', 'meaning': 'soldier / scholar '},
  {'zi': '彑', 'meaning': 'snout'},
  {'zi': '月', 'meaning': 'moon / meat'},
  {'zi': '玉', 'meaning': 'jade'},
  {'zi': '毛', 'meaning': 'fur / hair'},
  {'zi': '⺲', 'meaning': 'net'},
  {'zi': '用', 'meaning': 'use'},
  {'zi': '罒', 'meaning': 'net'},
  {'zi': '⺟', 'meaning': 'mother'},
  {'zi': '臣', 'meaning': 'minister'},
  {'zi': '曰', 'meaning': 'say'},
  {'zi': '鳥', 'meaning': 'bird'},
  {'zi': '穴', 'meaning': 'cave'},
  {'zi': '甩', 'meaning': 'discard'},
  {'zi': '亏', 'meaning': 'loss'},
  {'zi': '丑', 'meaning': 'ugly'},
  {'zi': '赱', 'meaning': 'run'},
  {'zi': '攴', 'meaning': ' tap / strike / rap'},
  {'zi': '鼠', 'meaning': 'rat'},
  {'zi': '玄', 'meaning': 'profound'},
  {'zi': '罓', 'meaning': 'net'},
  {'zi': '女', 'meaning': 'woman'},
  {'zi': '厂', 'meaning': 'cliff'},
  {'zi': '禸', 'meaning': 'track'},
  {'zi': '丿', 'meaning': 'slash'},
  {'zi': '鼓', 'meaning': 'drum'},
  {'zi': '豆', 'meaning': 'bean'},
  {'zi': '上', 'meaning': 'upper'},
  {'zi': '食', 'meaning': 'eat'},
  {'zi': '隹', 'meaning': 'short-tailed bird / good'},
  {'zi': '弓', 'meaning': 'bow / arrow'},
  {'zi': '身', 'meaning': 'body'},
    
    
  {'zi': '乡', 'meaning': 'village'},
  {'zi': '貝', 'meaning': 'shell'},
  {'zi': '目', 'meaning': 'eye'},
  {'zi': '旡', 'meaning': 'not'},
  {'zi': '囗', 'meaning': 'enclosure'},
  {'zi': '齐', 'meaning': 'neat'},
  {'zi': '土', 'meaning': 'earth'},
  {'zi': '门', 'meaning': ' door/gate'},
  {'zi': '邑', 'meaning': 'city (阝 right)'},
  {'zi': '力', 'meaning': 'strength / power'},
  {'zi': '冓', 'meaning': 'long'},
  {'zi': '文', 'meaning': 'writing / script'},
  {'zi': '川', 'meaning': 'river'},
  {'zi': '长', 'meaning': 'long'},
  {'zi': '乛', 'meaning': 'second'},
  {'zi': '钅', 'meaning': 'gold'},
  {'zi': '屮', 'meaning': 'sprout'},
  {'zi': '缶', 'meaning': 'jar'},
  {'zi': '羊', 'meaning': 'sheep'},
  {'zi': '纟', 'meaning': 'silk'},
  {'zi': '艸', 'meaning': 'grass'},
  {'zi': '革', 'meaning': 'leather'},
  {'zi': '而', 'meaning': 'and'},
  {'zi': '三', 'meaning': 'three / many'},
  {'zi': '鹿', 'meaning': 'deer'},
  {'zi': '黄', 'meaning': 'yellow'},
  {'zi': '手', 'meaning': 'hand'},
  {'zi': '襾', 'meaning': 'cover'}]

###     attrib = pinyin

In [29]:
data_pinyin = [] 
n_max = 1000  # 10  #  
n = 0
DEBUG = False # True #   
for r in radicals:
    if r == ' ': continue

    sql_5 = f"""
    with m as ( 
       
        select '{r}' as zi, pinyin
        from sh_kangxi_radical_214
        where trim(zi_radical) like '%{r}%'
        union 
        
        select '{r}' as zi, pinyin
        from sh_xiaoma
        where trim(zi_radical) like '%{r}%'
        union 
        
        select zi, pinyin
        from sh_w_radical_2
        where trim(zi) = '{r}'        
    )
    select * from m
    """
    df_5 = pd.read_sql(sql_5, conn)
    dd = df_5.to_dict("records")
    if dd: 
        data_pinyin += dd
        if DEBUG: 
            print(dd)
    
    n += 1 
    if n > n_max: break 

# len(data_pinyin), data_pinyin

result = process_zi_items(data_pinyin, name="pinyin")
data_pinyin_2 = [{'zi': k, 'pinyin':  " / ".join(v)} for k, v in result.items() if k != ' ']

len(data_pinyin_2), data_pinyin_2

(323,
 [{'zi': '豕', 'pinyin': 'shǐ'},
  {'zi': '夊', 'pinyin': 'suī'},
  {'zi': '⺷', 'pinyin': 'yáng'},
  {'zi': '刀', 'pinyin': 'dāo'},
  {'zi': '鼔', 'pinyin': 'gǔ'},
  {'zi': '車', 'pinyin': 'chē'},
  {'zi': '風', 'pinyin': 'fēng'},
  {'zi': '支', 'pinyin': 'zhī'},
  {'zi': '鬥', 'pinyin': 'dòu'},
  {'zi': '斗', 'pinyin': 'dǒu'},
  {'zi': '黑', 'pinyin': 'hēi'},
  {'zi': '欠', 'pinyin': 'qiàn'},
  {'zi': '⺍', 'pinyin': 'xiǎo'},
  {'zi': '⺮', 'pinyin': 'zhú'},
  {'zi': '襾', 'pinyin': 'xià / yà'},
  {'zi': '鼻', 'pinyin': 'bí'},
  {'zi': '龠', 'pinyin': 'yuè'},
  {'zi': '⺲', 'pinyin': 'wǎng'},
  {'zi': '方', 'pinyin': 'fāng'},
  {'zi': '邑', 'pinyin': 'yì'},
  {'zi': '彐', 'pinyin': 'jì / tiáo'},
  {'zi': '肉', 'pinyin': 'ròu'},
  {'zi': '竹', 'pinyin': 'zhú'},
  {'zi': '香', 'pinyin': 'xiāng'},
  {'zi': '几', 'pinyin': 'jī / jǐ'},
  {'zi': '川', 'pinyin': 'chuān'},
  {'zi': '辛', 'pinyin': 'xīn'},
  {'zi': '冖', 'pinyin': 'mì'},
  {'zi': '釒', 'pinyin': 'jīn'},
  {'zi': '厂', 'pinyin': 'hǎn[1] / hǎn'},
  {'

In [30]:
data_pinyin = [
    
    {'zi': '么', 'pinyin': 'yāo'},
  {'zi': '⺷', 'pinyin': 'yáng'},
  {'zi': '戶', 'pinyin': 'hù'},
  {'zi': '耒', 'pinyin': 'li / lěi'},
  {'zi': '彳', 'pinyin': 'chén / chì'},
  {'zi': '龠', 'pinyin': 'yuè'},
  {'zi': '魚', 'pinyin': 'yú'},
  {'zi': '疋', 'pinyin': 'pǐ / shū'},
  {'zi': '攵', 'pinyin': 'pū'},
  {'zi': '舟', 'pinyin': 'zhōu'},
  {'zi': '饣', 'pinyin': 'shí'},
  {'zi': '虍', 'pinyin': 'hū'},
  {'zi': '辵', 'pinyin': 'chuò'},
  {'zi': '阜', 'pinyin': 'fù'},
  {'zi': '亠', 'pinyin': 'tóu'},
  {'zi': '麻', 'pinyin': 'má'},
  {'zi': '赤', 'pinyin': 'chì'},
  {'zi': '⺻', 'pinyin': 'yù'},
  {'zi': '青', 'pinyin': 'qīng'},
  {'zi': '耳', 'pinyin': 'ěr'},
  {'zi': '見', 'pinyin': 'jiàn'},
  {'zi': '禾', 'pinyin': 'hé'},
  {'zi': '己', 'pinyin': 'jǐ'},
  {'zi': '丨', 'pinyin': 'gùn / gǔn'},
  {'zi': '辰', 'pinyin': 'chén'},
  {'zi': '入', 'pinyin': 'rù'},
  {'zi': '石', 'pinyin': 'shí'},
  {'zi': '儿', 'pinyin': 'rén / ér'},
  {'zi': '皮', 'pinyin': 'pí'},
  {'zi': '韋', 'pinyin': 'wéi'},
  {'zi': '匸', 'pinyin': 'xǐ / xǐ/xì'},
  {'zi': '雨', 'pinyin': 'yǔ'},
  {'zi': '讠', 'pinyin': 'yán'},
  {'zi': '十', 'pinyin': 'shí'},
  {'zi': '亻', 'pinyin': 'rén'},
  {'zi': '⺩', 'pinyin': 'yù'},
  {'zi': '车', 'pinyin': 'chē'},
  {'zi': '竹', 'pinyin': 'zhú'},
  {'zi': '疒', 'pinyin': 'chuáng / nè'},
  {'zi': '黽', 'pinyin': 'mǐn'},
  {'zi': '肀', 'pinyin': 'yù'},
  {'zi': '牜', 'pinyin': 'niú'},
  {'zi': '厶', 'pinyin': 'sī'},
  {'zi': '欠', 'pinyin': 'qiàn'},
  {'zi': '頁', 'pinyin': 'yè'},
  {'zi': '勹', 'pinyin': 'bāo'},
  {'zi': '虫', 'pinyin': 'chóng'},
  {'zi': '高', 'pinyin': 'gāo'},
  {'zi': '干', 'pinyin': 'gān'},
  {'zi': '耂', 'pinyin': 'lǎo'},
  {'zi': '癶', 'pinyin': 'bō'},
  {'zi': '斤', 'pinyin': 'jīn'},
  {'zi': '鼎', 'pinyin': 'dǐng'},
  {'zi': '⺪', 'pinyin': 'pǐ / shū'},
  {'zi': '比', 'pinyin': 'bǐ'},
  {'zi': '齿', 'pinyin': 'chǐ'},
  {'zi': '羽', 'pinyin': 'yǔ'},
  {'zi': '片', 'pinyin': 'piàn'},
  {'zi': '飞', 'pinyin': 'fēi'},
  {'zi': '自', 'pinyin': 'zì'},
  {'zi': '矢', 'pinyin': 'shǐ'},
  {'zi': '又', 'pinyin': 'yòu'},
  {'zi': '衤', 'pinyin': 'yī'},
  {'zi': '行', 'pinyin': 'xíng'},
  {'zi': '里', 'pinyin': 'lǐ'},
  {'zi': '巾', 'pinyin': 'jīn'},
  {'zi': '肉', 'pinyin': 'ròu'},
  {'zi': '乁', 'pinyin': 'piě'},
  {'zi': '丷', 'pinyin': 'bā'},
  {'zi': '小', 'pinyin': 'xiǎo'},
  {'zi': '口', 'pinyin': 'kǒu'},
  {'zi': '夊', 'pinyin': 'suī'},
  {'zi': '鬯', 'pinyin': 'chàng'},
  {'zi': '黾', 'pinyin': 'mǐn'},
  {'zi': '犭', 'pinyin': 'quǎn'},
  {'zi': '斉', 'pinyin': 'qí'},
  {'zi': '氺', 'pinyin': 'shuǐ'},
  {'zi': '生', 'pinyin': 'shēng'},
  {'zi': '糸', 'pinyin': 'mì'},
  {'zi': '金', 'pinyin': 'jīn'},
  {'zi': '首', 'pinyin': 'shǒu'},
  {'zi': '木', 'pinyin': 'mù'},
  {'zi': '西', 'pinyin': 'yà / xià'},
  {'zi': '麥', 'pinyin': 'mài'},
  {'zi': '犬', 'pinyin': 'quǎn'},
  {'zi': '飛', 'pinyin': 'fēi'},
  {'zi': '爫', 'pinyin': 'zhǎo'},
  {'zi': '⺺', 'pinyin': 'yù'},
  {'zi': '⺧', 'pinyin': 'niú'},
  {'zi': '色', 'pinyin': 'sè'},
  {'zi': '矛', 'pinyin': 'máo'},
  {'zi': '米', 'pinyin': 'mǐ'},
  {'zi': '舌', 'pinyin': 'shé'},
  {'zi': '鹵', 'pinyin': 'lǔ'},
  {'zi': '酉', 'pinyin': 'yǒu'},
  {'zi': '⺿', 'pinyin': 'cǎo'},
  {'zi': '鬥', 'pinyin': 'dòu'},
  {'zi': '由', 'pinyin': 'yóu'},
  {'zi': '牙', 'pinyin': 'yá'},
  {'zi': '母', 'pinyin': 'mǔ / wú'},
  {'zi': '扌', 'pinyin': 'shǒu / bǎ'},
  {'zi': '匕', 'pinyin': 'bǐ'},
  {'zi': '斗', 'pinyin': 'dǒu'},
  {'zi': '訁', 'pinyin': 'yán'},
  {'zi': '鬼', 'pinyin': 'guǐ'},
  {'zi': '車', 'pinyin': 'chē'},
  {'zi': '⻖', 'pinyin': 'fù'},
  {'zi': '風', 'pinyin': 'fēng'},
  {'zi': '歺', 'pinyin': 'dǎi'},
  {'zi': '网', 'pinyin': 'wǎng'},
  {'zi': '尢', 'pinyin': 'wāng'},
  {'zi': '匚', 'pinyin': 'fāng'},
  {'zi': '卩', 'pinyin': 'jié'},
  {'zi': '马', 'pinyin': 'mǎ'},
  {'zi': '子', 'pinyin': 'zǐ'},
  {'zi': '辛', 'pinyin': 'xīn'},
  {'zi': '⻌', 'pinyin': 'chuò'},
  {'zi': '龙', 'pinyin': 'lóng'},
  {'zi': '无', 'pinyin': 'wú'},
  {'zi': '黑', 'pinyin': 'hēi'},
  {'zi': '龍', 'pinyin': 'lóng'},
  {'zi': '田', 'pinyin': 'tián'},
  {'zi': '音', 'pinyin': 'yīn'},
  {'zi': '長', 'pinyin': 'cháng'},
  {'zi': '糹', 'pinyin': 'mì'},
  {'zi': '⺍', 'pinyin': 'xiǎo'},
  {'zi': '⺳', 'pinyin': 'wǎng'},
  {'zi': '臼', 'pinyin': 'jiù'},
  {'zi': '乚', 'pinyin': 'yǐ'},
  {'zi': '㔾', 'pinyin': 'jié'},
  {'zi': '尣', 'pinyin': 'wāng'},
  {'zi': '香', 'pinyin': 'xiāng'},
  {'zi': '鱼', 'pinyin': 'yú'},
  {'zi': '巜', 'pinyin': 'chuān'},
  {'zi': '殳', 'pinyin': 'shū'},
  {'zi': '戈', 'pinyin': 'gē'},
  {'zi': '氵', 'pinyin': 'shuǐ'},
  {'zi': '鸟', 'pinyin': 'niǎo'},
  {'zi': '鬲', 'pinyin': 'lì'},
  {'zi': '釆', 'pinyin': 'biàn'},
  {'zi': '凵', 'pinyin': 'qū / qǔ / kǎn'},
  {'zi': '走', 'pinyin': 'zǒu'},
  {'zi': '刂', 'pinyin': 'dāo / biāo'},
  {'zi': '丶', 'pinyin': 'zhǔ'},
  {'zi': '隶', 'pinyin': 'lì / dài'},
  {'zi': '卜', 'pinyin': 'bǔ'},
  {'zi': '方', 'pinyin': 'fāng'},
  {'zi': '瓜', 'pinyin': 'guā'},
  {'zi': '彐', 'pinyin': 'jì / tiáo'},
  {'zi': '二', 'pinyin': 'ér / èr'},
  {'zi': '非', 'pinyin': 'fēi'},
  {'zi': '几', 'pinyin': 'jī / jǐ'},
  {'zi': '皿', 'pinyin': 'mǐn'},
  {'zi': '王', 'pinyin': 'yù'},
  {'zi': '牛', 'pinyin': 'niú'},
  {'zi': '冫', 'pinyin': 'bīng'},
  {'zi': '风', 'pinyin': 'fēng'},
  {'zi': '⺼', 'pinyin': 'ròu'},
  {'zi': '⺌', 'pinyin': 'xiǎo'},
  {'zi': '爿', 'pinyin': 'qiáng'},
  {'zi': '白', 'pinyin': 'bái'},
  {'zi': '页', 'pinyin': 'yè'},
  {'zi': '鼔', 'pinyin': 'gǔ'},
  {'zi': '足', 'pinyin': 'zú'},
  {'zi': '⻊', 'pinyin': 'zú'},
  {'zi': '忄', 'pinyin': 'xīn'},
  {'zi': '乙', 'pinyin': 'yǐ'},
  {'zi': '鼻', 'pinyin': 'bí'},
  {'zi': '龟', 'pinyin': 'guī'},
  {'zi': '氏', 'pinyin': 'shì'},
  {'zi': '贝', 'pinyin': 'bèi'},
  {'zi': '广', 'pinyin': 'guǎng / yǎn'},
  {'zi': '灬', 'pinyin': 'huǒ'},
  {'zi': '日', 'pinyin': 'rì'},
  {'zi': '龜', 'pinyin': 'guī'},
  {'zi': '八', 'pinyin': 'bā'},
  {'zi': '黹', 'pinyin': 'zhǐ'},
  {'zi': '廴', 'pinyin': 'yǐn'},
  {'zi': '立', 'pinyin': 'lì'},
  {'zi': '衣', 'pinyin': 'yī'},
  {'zi': '黃', 'pinyin': 'huáng'},
  {'zi': '工', 'pinyin': 'gōng'},
  {'zi': '⺫', 'pinyin': 'mù'},
  {'zi': '大', 'pinyin': 'dà'},
  {'zi': '一', 'pinyin': 'yī'},
  {'zi': '⺶', 'pinyin': 'yáng'},
  {'zi': '髙', 'pinyin': 'gāo'},
  {'zi': '宀', 'pinyin': 'mián'},
  {'zi': '⻂', 'pinyin': 'yī'},
  {'zi': '乀', 'pinyin': 'piě'},
  {'zi': '止', 'pinyin': 'zhǐ'},
  {'zi': '⺮', 'pinyin': 'zhú'},
  {'zi': '彡', 'pinyin': 'shān'},
  {'zi': '黍', 'pinyin': 'shǔ'},
  {'zi': '冂', 'pinyin': 'jiǒng / jiōng'},
  {'zi': '㣺', 'pinyin': 'xīn'},
  {'zi': '齒', 'pinyin': 'chǐ'},
  {'zi': '飠', 'pinyin': 'shí'},
  {'zi': '⻏', 'pinyin': 'yì'},
  {'zi': '支', 'pinyin': 'zhī'},
  {'zi': '玊', 'pinyin': 'yù'},
  {'zi': '龵', 'pinyin': 'shǒu'},
  {'zi': '夂', 'pinyin': 'zhǐ'},
  {'zi': '瓦', 'pinyin': 'wǎ'},
  {'zi': '幺', 'pinyin': 'yāo'},
  {'zi': '戸', 'pinyin': 'hù'},
  {'zi': '毋', 'pinyin': 'mǔ / wú'},
  {'zi': '甘', 'pinyin': 'gān'},
  {'zi': '亅', 'pinyin': 'gōu / jué'},
  {'zi': '靑', 'pinyin': 'qīng'},
  {'zi': '舛', 'pinyin': 'chuǎn'},
  {'zi': '至', 'pinyin': 'zhì'},
  {'zi': '髟', 'pinyin': 'biāo'},
  {'zi': '門', 'pinyin': 'mén'},
  {'zi': '山', 'pinyin': 'shān'},
  {'zi': '面', 'pinyin': 'miàn'},
  {'zi': '卤', 'pinyin': 'lǔ'},
  {'zi': '爪', 'pinyin': 'zhǎo'},
  {'zi': '聿', 'pinyin': 'yù'},
  {'zi': '豸', 'pinyin': 'zhì'},
  {'zi': '廾', 'pinyin': 'gǒng'},
  {'zi': '父', 'pinyin': 'fù'},
  {'zi': '镸', 'pinyin': 'cháng'},
  {'zi': '刀', 'pinyin': 'dāo'},
  {'zi': '⻎', 'pinyin': 'chuò'},
  {'zi': '韦', 'pinyin': 'wéi'},
  {'zi': '见', 'pinyin': 'jiàn'},
  {'zi': '⺗', 'pinyin': 'xīn'},
  {'zi': '艹', 'pinyin': 'cǎo'},
  {'zi': '夕', 'pinyin': 'xī'},
  {'zi': '阝', 'pinyin': 'fù / yì'},
  {'zi': '冖', 'pinyin': 'mì'},
  {'zi': '辶', 'pinyin': 'chuò'},
  {'zi': '丬', 'pinyin': 'qiáng'},
  {'zi': '⺄', 'pinyin': 'piě'},
  {'zi': '靣', 'pinyin': 'miàn'},
  {'zi': '⻇', 'pinyin': 'jiǎo'},
  {'zi': '礻', 'pinyin': 'shì'},
  {'zi': '水', 'pinyin': 'shuǐ'},
  {'zi': '馬', 'pinyin': 'mǎ'},
  {'zi': '言', 'pinyin': 'yán'},
  {'zi': '心', 'pinyin': 'xīn'},
  {'zi': '爻', 'pinyin': 'yáo'},
  {'zi': '覀', 'pinyin': 'yà / xià'},
  {'zi': '韭', 'pinyin': 'jiǔ'},
  {'zi': '齊', 'pinyin': 'qí'},
  {'zi': '艮', 'pinyin': 'gèn'},
  {'zi': '⻍', 'pinyin': 'chuò'},
  {'zi': '弋', 'pinyin': 'yì'},
  {'zi': '角', 'pinyin': 'jué / jiǎo'},
  {'zi': '尸', 'pinyin': 'shī'},
  {'zi': '釒', 'pinyin': 'jīn'},
  {'zi': '歹', 'pinyin': 'dǎi'},
  {'zi': '谷', 'pinyin': 'gǔ'},
  {'zi': '老', 'pinyin': 'lǎo'},
  {'zi': '豕', 'pinyin': 'shǐ'},
  {'zi': '⺈', 'pinyin': 'dāo'},
  {'zi': '血', 'pinyin': 'xiě / xuè'},
  {'zi': '户', 'pinyin': 'hù'},
  {'zi': '麦', 'pinyin': 'mài'},
  {'zi': '示', 'pinyin': 'shì'},
  {'zi': '火', 'pinyin': 'huǒ'},
  {'zi': '人', 'pinyin': 'rén'},
  {'zi': '巛', 'pinyin': 'chuān'},
  {'zi': '骨', 'pinyin': 'gǔ'},
  {'zi': '气', 'pinyin': 'qì'},
  {'zi': '寸', 'pinyin': 'cùn'},
  {'zi': '士', 'pinyin': 'shì'},
  {'zi': '彑', 'pinyin': 'jì'},
  {'zi': '月', 'pinyin': 'ròu / yuè'},
  {'zi': '玉', 'pinyin': 'yù'},
  {'zi': '毛', 'pinyin': 'máo'},
  {'zi': '⺲', 'pinyin': 'wǎng'},
  {'zi': '用', 'pinyin': 'yòng'},
  {'zi': '罒', 'pinyin': 'wǎng'},
  {'zi': '⺟', 'pinyin': 'mǔ'},
  {'zi': '臣', 'pinyin': 'chén'},
  {'zi': '曰', 'pinyin': 'yuē'},
  {'zi': '鳥', 'pinyin': 'niǎo'},
  {'zi': '穴', 'pinyin': 'xué'},
  {'zi': '甩', 'pinyin': 'yòng'},
  {'zi': '丑', 'pinyin': 'chǒu'},
  {'zi': '赱', 'pinyin': 'zǒu'},
  {'zi': '攴', 'pinyin': 'pū / pō'},
  {'zi': '鼠', 'pinyin': 'shǔ'},
  {'zi': '玄', 'pinyin': 'xuán'},
  {'zi': '罓', 'pinyin': 'wǎng'},
  {'zi': '女', 'pinyin': 'nǚ'},
  {'zi': '厂', 'pinyin': 'hǎn[1] / hǎn'},
  {'zi': '禸', 'pinyin': 'róu'},
  {'zi': '丿', 'pinyin': 'piě'},
  {'zi': '鼓', 'pinyin': 'gǔ'},
  {'zi': '豆', 'pinyin': 'dòu'},
  {'zi': '食', 'pinyin': 'shí'},
  {'zi': '隹', 'pinyin': 'zhuī'},
  {'zi': '弓', 'pinyin': 'gōng'},
  {'zi': '身', 'pinyin': 'shēn'},
  {'zi': '貝', 'pinyin': 'bèi'},
  {'zi': '目', 'pinyin': 'mù'},
  {'zi': '旡', 'pinyin': 'wú'},
  {'zi': '囗', 'pinyin': 'wāng / wéi'},
  {'zi': '齐', 'pinyin': 'qí'},
  {'zi': '土', 'pinyin': 'tǔ'},
  {'zi': '门', 'pinyin': 'mén'},
  {'zi': '邑', 'pinyin': 'yì'},
  {'zi': '力', 'pinyin': 'lì / li'},
  {'zi': '冓', 'pinyin': 'yǒng'},
  {'zi': '文', 'pinyin': 'wén'},
  {'zi': '川', 'pinyin': 'chuān'},
  {'zi': '长', 'pinyin': 'cháng'},
  {'zi': '乛', 'pinyin': 'yǐ'},
  {'zi': '钅', 'pinyin': 'jīn'},
  {'zi': '屮', 'pinyin': 'chè'},
  {'zi': '缶', 'pinyin': 'fǒu'},
  {'zi': '羊', 'pinyin': 'yáng'},
  {'zi': '纟', 'pinyin': 'mì'},
  {'zi': '艸', 'pinyin': 'cǎo'},
  {'zi': '革', 'pinyin': 'gé'},
  {'zi': '而', 'pinyin': 'ér'},
  {'zi': '鹿', 'pinyin': 'lù'},
  {'zi': '黄', 'pinyin': 'huáng'},
  {'zi': '手', 'pinyin': 'shǒu'},
  {'zi': '襾', 'pinyin': 'yà / xià'}]

###     attrib = n_strokes

In [31]:
data_n_strokes = [] 
n_max = 1000  # 10  #   
n = 0
DEBUG = False # True #   
for r in radicals:
    if r == ' ': continue

    sql_5 = f"""
    with m as ( 
       
        select '{r}' as zi, n_strokes
        from sh_kangxi_radical_214
        where trim(zi_radical) like '%{r}%'
        union 
        
        select '{r}' as zi, n_strokes
        from sh_xiaoma
        where trim(zi_radical) like '%{r}%'
        union 
        
        select zi, n_strokes
        from sh_elemental_zi_v2
        where trim(zi) = '{r}'        
    )
    select * from m
    """
    df_5 = pd.read_sql(sql_5, conn)
    dd = df_5.to_dict("records")
    if dd: 
        data_n_strokes += dd
        if DEBUG: 
            print(dd)
    
    n += 1 
    if n > n_max: break 

# len(data_n_strokes), data_n_strokes

result = process_zi_items(data_n_strokes, name="n_strokes")
data_n_strokes_2 = [{'zi': k, 'n_strokes':  " / ".join(v)} for k, v in result.items() if k != ' ']

len(data_n_strokes_2), data_n_strokes_2

(490,
 [{'zi': '豕', 'n_strokes': '7'},
  {'zi': '夊', 'n_strokes': '3'},
  {'zi': '⺷', 'n_strokes': '6'},
  {'zi': '刀', 'n_strokes': '2'},
  {'zi': '鼔', 'n_strokes': '13'},
  {'zi': '車', 'n_strokes': '7'},
  {'zi': '乐', 'n_strokes': '5'},
  {'zi': '風', 'n_strokes': '4 / 9'},
  {'zi': '支', 'n_strokes': '4'},
  {'zi': '鬥', 'n_strokes': '10'},
  {'zi': '斗', 'n_strokes': '4'},
  {'zi': '黑', 'n_strokes': '12'},
  {'zi': '欠', 'n_strokes': '4'},
  {'zi': '⺍', 'n_strokes': '3'},
  {'zi': '本', 'n_strokes': '5'},
  {'zi': '⺮', 'n_strokes': '6'},
  {'zi': '襾', 'n_strokes': '6'},
  {'zi': '鼻', 'n_strokes': '14'},
  {'zi': '开', 'n_strokes': '4'},
  {'zi': '龠', 'n_strokes': '17'},
  {'zi': '⺲', 'n_strokes': '6'},
  {'zi': '内', 'n_strokes': '4'},
  {'zi': '方', 'n_strokes': '4'},
  {'zi': '邑', 'n_strokes': '7'},
  {'zi': '彐', 'n_strokes': '3'},
  {'zi': '九', 'n_strokes': '2'},
  {'zi': '肉', 'n_strokes': '6'},
  {'zi': '竹', 'n_strokes': '6'},
  {'zi': '尧', 'n_strokes': '6'},
  {'zi': '香', 'n_strokes': '

In [32]:
data_n_strokes = [{'zi': '么', 'n_strokes': '3'},
  {'zi': '以', 'n_strokes': '4'},
  {'zi': '⺷', 'n_strokes': '6'},
  {'zi': '丈', 'n_strokes': '3'},
  {'zi': '戶', 'n_strokes': '4'},
  {'zi': '耒', 'n_strokes': '6'},
  {'zi': '彳', 'n_strokes': '3'},
  {'zi': '龠', 'n_strokes': '17'},
  {'zi': '东', 'n_strokes': '5'},
  {'zi': '魚', 'n_strokes': '11'},
  {'zi': '疋', 'n_strokes': '5'},
  {'zi': '攵', 'n_strokes': '4'},
  {'zi': '舟', 'n_strokes': '6'},
  {'zi': '乂', 'n_strokes': '2'},
  {'zi': '饣', 'n_strokes': '3'},
  {'zi': '虍', 'n_strokes': '6'},
  {'zi': '刍', 'n_strokes': '5'},
  {'zi': '兄', 'n_strokes': '5'},
  {'zi': '辵', 'n_strokes': '7'},
  {'zi': '阜', 'n_strokes': '8'},
  {'zi': '⺀', 'n_strokes': '2'},
  {'zi': '亠', 'n_strokes': '2'},
  {'zi': '麻', 'n_strokes': '11'},
  {'zi': '赤', 'n_strokes': '7'},
  {'zi': '⺻', 'n_strokes': '6'},
  {'zi': '青', 'n_strokes': '8'},
  {'zi': '耳', 'n_strokes': '6'},
  {'zi': '㐬', 'n_strokes': '7'},
  {'zi': '見', 'n_strokes': '7'},
  {'zi': '禾', 'n_strokes': '5'},
  {'zi': '丙', 'n_strokes': '5'},
  {'zi': '己', 'n_strokes': '3'},
  {'zi': '丨', 'n_strokes': '1'},
  {'zi': '吉', 'n_strokes': '6'},
  {'zi': '辰', 'n_strokes': '7'},
  {'zi': '入', 'n_strokes': '2'},
  {'zi': '可', 'n_strokes': '5'},
  {'zi': '屯', 'n_strokes': '4'},
  {'zi': '巴', 'n_strokes': '4'},
  {'zi': '石', 'n_strokes': '5'},
  {'zi': '儿', 'n_strokes': '2'},
  {'zi': '皮', 'n_strokes': '5'},
  {'zi': '韋', 'n_strokes': '9'},
  {'zi': '匸', 'n_strokes': '2'},
  {'zi': '雨', 'n_strokes': '8'},
  {'zi': '讠', 'n_strokes': '2'},
  {'zi': '十', 'n_strokes': '2'},
  {'zi': '亻', 'n_strokes': '2'},
  {'zi': '⺩', 'n_strokes': '4'},
  {'zi': '车', 'n_strokes': '4'},
  {'zi': '竹', 'n_strokes': '6'},
  {'zi': '疒', 'n_strokes': '5'},
  {'zi': '平', 'n_strokes': '5'},
  {'zi': '奉', 'n_strokes': '8'},
  {'zi': '黽', 'n_strokes': '13'},
  {'zi': '肀', 'n_strokes': '6'},
  {'zi': '牜', 'n_strokes': '4'},
  {'zi': '厶', 'n_strokes': '2'},
  {'zi': '其', 'n_strokes': '8'},
  {'zi': '欠', 'n_strokes': '4'},
  {'zi': '頁', 'n_strokes': '9'},
  {'zi': '勹', 'n_strokes': '2'},
  {'zi': '虫', 'n_strokes': '6'},
  {'zi': '高', 'n_strokes': '10'},
  {'zi': '干', 'n_strokes': '3'},
  {'zi': '耂', 'n_strokes': '4'},
  {'zi': '癶', 'n_strokes': '5'},
  {'zi': '斤', 'n_strokes': '4'},
  {'zi': '鼎', 'n_strokes': '13'},
  {'zi': '⺪', 'n_strokes': '5'},
  {'zi': '比', 'n_strokes': '5'},
  {'zi': '乍', 'n_strokes': '5'},
  {'zi': '业', 'n_strokes': '5'},
  {'zi': '齿', 'n_strokes': '8'},
  {'zi': '羽', 'n_strokes': '6'},
  {'zi': '另', 'n_strokes': '5'},
  {'zi': '坙', 'n_strokes': '7'},
  {'zi': '㕻', 'n_strokes': '7'},
  {'zi': '片', 'n_strokes': '4'},
  {'zi': '甲', 'n_strokes': '5'},
  {'zi': '飞', 'n_strokes': '3'},
  {'zi': '自', 'n_strokes': '6'},
  {'zi': '才', 'n_strokes': '3'},
  {'zi': '矢', 'n_strokes': '5'},
  {'zi': '又', 'n_strokes': '2'},
  {'zi': '衤', 'n_strokes': '5'},
  {'zi': '行', 'n_strokes': '6'},
  {'zi': '𤴔', 'n_strokes': '5'},
  {'zi': '央', 'n_strokes': '5'},
  {'zi': '少', 'n_strokes': '4'},
  {'zi': '里', 'n_strokes': '7'},
  {'zi': '句', 'n_strokes': '5'},
  {'zi': '万', 'n_strokes': '3'},
  {'zi': '屰', 'n_strokes': '6'},
  {'zi': '巾', 'n_strokes': '3'},
  {'zi': '令', 'n_strokes': '5'},
  {'zi': '肉', 'n_strokes': '6'},
  {'zi': '乁', 'n_strokes': '1'},
  {'zi': '丷', 'n_strokes': '2'},
  {'zi': '小', 'n_strokes': '3'},
  {'zi': '丘', 'n_strokes': '5'},
  {'zi': '口', 'n_strokes': '3'},
  {'zi': '于', 'n_strokes': '3'},
  {'zi': '夊', 'n_strokes': '3'},
  {'zi': '未', 'n_strokes': '5'},
  {'zi': '鬯', 'n_strokes': '10'},
  {'zi': '仓', 'n_strokes': '4'},
  {'zi': '黾', 'n_strokes': '8'},
  {'zi': '犭', 'n_strokes': '3'},
  {'zi': '斉', 'n_strokes': '14'},
  {'zi': '囟', 'n_strokes': '6'},
  {'zi': '并', 'n_strokes': '6'},
  {'zi': '氺', 'n_strokes': '5'},
  {'zi': '生', 'n_strokes': '5'},
  {'zi': '亡', 'n_strokes': '3'},
  {'zi': '北', 'n_strokes': '5'},
  {'zi': '糸', 'n_strokes': '6'},
  {'zi': '金', 'n_strokes': '8'},
  {'zi': '首', 'n_strokes': '9'},
  {'zi': '木', 'n_strokes': '4'},
  {'zi': '今', 'n_strokes': '4'},
  {'zi': '西', 'n_strokes': '6'},
  {'zi': '麥', 'n_strokes': '11'},
  {'zi': '犬', 'n_strokes': '4'},
  {'zi': '飛', 'n_strokes': '9'},
  {'zi': '爫', 'n_strokes': '4'},
  {'zi': '冈', 'n_strokes': '4'},
  {'zi': '⺺', 'n_strokes': '6'},
  {'zi': '⺧', 'n_strokes': '4'},
  {'zi': '布', 'n_strokes': '5'},
  {'zi': '色', 'n_strokes': '6'},
  {'zi': '电', 'n_strokes': '5'},
  {'zi': '矛', 'n_strokes': '5'},
  {'zi': '刁', 'n_strokes': '2'},
  {'zi': '米', 'n_strokes': '6'},
  {'zi': '舌', 'n_strokes': '6'},
  {'zi': '鹵', 'n_strokes': '11'},
  {'zi': '酉', 'n_strokes': '7'},
  {'zi': '⺿', 'n_strokes': '6'},
  {'zi': '鬥', 'n_strokes': '10'},
  {'zi': '由', 'n_strokes': '5'},
  {'zi': '冬', 'n_strokes': '5'},
  {'zi': '牙', 'n_strokes': '4'},
  {'zi': '亍', 'n_strokes': '3'},
  {'zi': '个', 'n_strokes': '3'},
  {'zi': '母', 'n_strokes': '5'},
  {'zi': '扌', 'n_strokes': '3'},
  {'zi': '孚', 'n_strokes': '7'},
  {'zi': '匕', 'n_strokes': '2'},
  {'zi': '术', 'n_strokes': '5'},
  {'zi': '斗', 'n_strokes': '4'},
  {'zi': '訁', 'n_strokes': '7'},
  {'zi': '旦', 'n_strokes': '5'},
  {'zi': '习', 'n_strokes': '3'},
  {'zi': '尺', 'n_strokes': '4'},
  {'zi': '鬼', 'n_strokes': '9'},
  {'zi': '畐', 'n_strokes': '9'},
  {'zi': '車', 'n_strokes': '7'},
  {'zi': '⻖', 'n_strokes': '8'},
  {'zi': '勾', 'n_strokes': '4'},
  {'zi': '丰', 'n_strokes': '4'},
  {'zi': '風', 'n_strokes': '9'},
  {'zi': '歺', 'n_strokes': '4'},
  {'zi': '网', 'n_strokes': '6'},
  {'zi': '云', 'n_strokes': '4'},
  {'zi': '尢', 'n_strokes': '3'},
  {'zi': '勿', 'n_strokes': '4'},
  {'zi': '匚', 'n_strokes': '2'},
  {'zi': '卩', 'n_strokes': '2'},
  {'zi': '马', 'n_strokes': '3'},
  {'zi': '子', 'n_strokes': '3'},
  {'zi': '辛', 'n_strokes': '7'},
  {'zi': '⻌', 'n_strokes': '7'},
  {'zi': '龙', 'n_strokes': '5'},
  {'zi': '无', 'n_strokes': '4'},
  {'zi': '黑', 'n_strokes': '12'},
  {'zi': '宁', 'n_strokes': '5'},
  {'zi': '龍', 'n_strokes': '16'},
  {'zi': '田', 'n_strokes': '5'},
  {'zi': '音', 'n_strokes': '9'},
  {'zi': '長', 'n_strokes': '8'},
  {'zi': '庄', 'n_strokes': '6'},
  {'zi': '糹', 'n_strokes': '6'},
  {'zi': '⺍', 'n_strokes': '3'},
  {'zi': '⺳', 'n_strokes': '6'},
  {'zi': '臼', 'n_strokes': '6'},
  {'zi': '乚', 'n_strokes': '1'},
  {'zi': '㔾', 'n_strokes': '2'},
  {'zi': '尣', 'n_strokes': '3'},
  {'zi': '香', 'n_strokes': '9'},
  {'zi': '鱼', 'n_strokes': '8'},
  {'zi': '巜', 'n_strokes': '3'},
  {'zi': '殳', 'n_strokes': '4'},
  {'zi': '戈', 'n_strokes': '4'},
  {'zi': '氵', 'n_strokes': '3'},
  {'zi': '它', 'n_strokes': '5'},
  {'zi': '鸟', 'n_strokes': '5'},
  {'zi': '鬲', 'n_strokes': '10'},
  {'zi': '釆', 'n_strokes': '7'},
  {'zi': '凵', 'n_strokes': '2'},
  {'zi': '走', 'n_strokes': '7'},
  {'zi': '刂', 'n_strokes': '2'},
  {'zi': '丶', 'n_strokes': '1'},
  {'zi': '不', 'n_strokes': '4'},
  {'zi': '击', 'n_strokes': '5'},
  {'zi': '交', 'n_strokes': '6'},
  {'zi': '乐', 'n_strokes': '5'},
  {'zi': '肙', 'n_strokes': '7'},
  {'zi': '隶', 'n_strokes': '8'},
  {'zi': '卜', 'n_strokes': '2'},
  {'zi': '久', 'n_strokes': '3'},
  {'zi': '方', 'n_strokes': '4'},
  {'zi': '瓜', 'n_strokes': '5'},
  {'zi': '彐', 'n_strokes': '3'},
  {'zi': '奂', 'n_strokes': '7'},
  {'zi': '二', 'n_strokes': '2'},
  {'zi': '非', 'n_strokes': '8'},
  {'zi': '几', 'n_strokes': '2'},
  {'zi': '皿', 'n_strokes': '5'},
  {'zi': '王', 'n_strokes': '4'},
  {'zi': '刃', 'n_strokes': '3'},
  {'zi': '牛', 'n_strokes': '4'},
  {'zi': '冫', 'n_strokes': '2'},
  {'zi': '风', 'n_strokes': '4'},
  {'zi': '中', 'n_strokes': '4'},
  {'zi': '之', 'n_strokes': '3'},
  {'zi': '⺼', 'n_strokes': '6'},
  {'zi': '⺌', 'n_strokes': '3'},
  {'zi': '爿', 'n_strokes': '4'},
  {'zi': '白', 'n_strokes': '5'},
  {'zi': '页', 'n_strokes': '6'},
  {'zi': '鼔', 'n_strokes': '13'},
  {'zi': '足', 'n_strokes': '7'},
  {'zi': '⻊', 'n_strokes': '7'},
  {'zi': '忄', 'n_strokes': '3'},
  {'zi': '乙', 'n_strokes': '1'},
  {'zi': '鼻', 'n_strokes': '14'},
  {'zi': '龟', 'n_strokes': '7'},
  {'zi': '柬', 'n_strokes': '9'},
  {'zi': '氏', 'n_strokes': '4'},
  {'zi': '贝', 'n_strokes': '4'},
  {'zi': '广', 'n_strokes': '3'},
  {'zi': '灬', 'n_strokes': '4'},
  {'zi': '日', 'n_strokes': '4'},
  {'zi': '龜', 'n_strokes': '16'},
  {'zi': '八', 'n_strokes': '2'},
  {'zi': '黹', 'n_strokes': '12'},
  {'zi': '廴', 'n_strokes': '3'},
  {'zi': '立', 'n_strokes': '5'},
  {'zi': '衣', 'n_strokes': '6'},
  {'zi': '为', 'n_strokes': '4'},
  {'zi': '四', 'n_strokes': '5'},
  {'zi': '黃', 'n_strokes': '12'},
  {'zi': '工', 'n_strokes': '3'},
  {'zi': '⺫', 'n_strokes': '5'},
  {'zi': '九', 'n_strokes': '2'},
  {'zi': '大', 'n_strokes': '3'},
  {'zi': '友', 'n_strokes': '4'},
  {'zi': '一', 'n_strokes': '1'},
  {'zi': '⺶', 'n_strokes': '6'},
  {'zi': '髙', 'n_strokes': '10'},
  {'zi': '凶', 'n_strokes': '4'},
  {'zi': '匀', 'n_strokes': '4'},
  {'zi': '宀', 'n_strokes': '3'},
  {'zi': '⻂', 'n_strokes': '6'},
  {'zi': '乀', 'n_strokes': '1'},
  {'zi': '止', 'n_strokes': '4'},
  {'zi': '乃', 'n_strokes': '2'},
  {'zi': '東', 'n_strokes': '8'},
  {'zi': '⺮', 'n_strokes': '6'},
  {'zi': '千', 'n_strokes': '3'},
  {'zi': '彡', 'n_strokes': '3'},
  {'zi': '六', 'n_strokes': '4'},
  {'zi': '黍', 'n_strokes': '12'},
  {'zi': '尚', 'n_strokes': '8'},
  {'zi': '禺 ', 'n_strokes': '9'},
  {'zi': '冂', 'n_strokes': '2'},
  {'zi': '向', 'n_strokes': '6'},
  {'zi': '各', 'n_strokes': '6'},
  {'zi': '㣺', 'n_strokes': '4'},
  {'zi': '齒', 'n_strokes': '15'},
  {'zi': '飠', 'n_strokes': '9'},
  {'zi': '⻏', 'n_strokes': '3'},
  {'zi': '办', 'n_strokes': '4'},
  {'zi': '支', 'n_strokes': '4'},
  {'zi': '与', 'n_strokes': '3'},
  {'zi': '义', 'n_strokes': '3'},
  {'zi': '寺', 'n_strokes': '6'},
  {'zi': '玊', 'n_strokes': '5'},
  {'zi': '龵', 'n_strokes': '4'},
  {'zi': '夂', 'n_strokes': '3'},
  {'zi': '且', 'n_strokes': '5'},
  {'zi': '瓦', 'n_strokes': '4'},
  {'zi': '下', 'n_strokes': '3'},
  {'zi': '幺', 'n_strokes': '3'},
  {'zi': '戸', 'n_strokes': '4'},
  {'zi': '申', 'n_strokes': '5'},
  {'zi': '毋', 'n_strokes': '4'},
  {'zi': '勺', 'n_strokes': '3'},
  {'zi': '甘', 'n_strokes': '5'},
  {'zi': '亅', 'n_strokes': '1'},
  {'zi': '靑', 'n_strokes': '8'},
  {'zi': '舛', 'n_strokes': '7'},
  {'zi': '至', 'n_strokes': '6'},
  {'zi': '髟', 'n_strokes': '10'},
  {'zi': '公', 'n_strokes': '4'},
  {'zi': '門', 'n_strokes': '8'},
  {'zi': '夆', 'n_strokes': '7'},
  {'zi': '山', 'n_strokes': '3'},
  {'zi': '太', 'n_strokes': '4'},
  {'zi': '面', 'n_strokes': '9'},
  {'zi': '曲', 'n_strokes': '6'},
  {'zi': '七', 'n_strokes': '2'},
  {'zi': '卤', 'n_strokes': '7'},
  {'zi': '爪', 'n_strokes': '4'},
  {'zi': '聿', 'n_strokes': '6'},
  {'zi': '豸', 'n_strokes': '7'},
  {'zi': '廾', 'n_strokes': '3'},
  {'zi': '父', 'n_strokes': '4'},
  {'zi': '镸', 'n_strokes': '8'},
  {'zi': '已', 'n_strokes': '3'},
  {'zi': '刀', 'n_strokes': '2'},
  {'zi': '⻎', 'n_strokes': '7'},
  {'zi': '韦', 'n_strokes': '4'},
  {'zi': '甫', 'n_strokes': '7'},
  {'zi': '及', 'n_strokes': '3'},
  {'zi': '见', 'n_strokes': '4'},
  {'zi': '市', 'n_strokes': '5'},
  {'zi': '⺗', 'n_strokes': '4'},
  {'zi': '吕', 'n_strokes': '6'},
  {'zi': '艹', 'n_strokes': '3'},
  {'zi': '尧', 'n_strokes': '6'},
  {'zi': '古', 'n_strokes': '5'},
  {'zi': '夕', 'n_strokes': '3'},
  {'zi': '阝', 'n_strokes': '3'},
  {'zi': '冖', 'n_strokes': '2'},
  {'zi': '乞', 'n_strokes': '3'},
  {'zi': '辶', 'n_strokes': '3'},
  {'zi': '丬', 'n_strokes': '3'},
  {'zi': '⺄', 'n_strokes': '1'},
  {'zi': '靣', 'n_strokes': '9'},
  {'zi': '午', 'n_strokes': '4'},
  {'zi': '⻇', 'n_strokes': '7'},
  {'zi': '员', 'n_strokes': '7'},
  {'zi': '礻', 'n_strokes': '4'},
  {'zi': '水', 'n_strokes': '4'},
  {'zi': '馬', 'n_strokes': '10'},
  {'zi': '言', 'n_strokes': '7'},
  {'zi': '心', 'n_strokes': '4'},
  {'zi': '也', 'n_strokes': '3'},
  {'zi': '爻', 'n_strokes': '4'},
  {'zi': '覀', 'n_strokes': '6'},
  {'zi': '韭', 'n_strokes': '9'},
  {'zi': '齊', 'n_strokes': '14'},
  {'zi': '良', 'n_strokes': '7'},
  {'zi': '乌', 'n_strokes': '4'},
  {'zi': '丸', 'n_strokes': '3'},
  {'zi': '艮', 'n_strokes': '6'},
  {'zi': '区', 'n_strokes': '4'},
  {'zi': '⻍', 'n_strokes': '7'},
  {'zi': '丁', 'n_strokes': '2'},
  {'zi': '弋', 'n_strokes': '3'},
  {'zi': '必', 'n_strokes': '5'},
  {'zi': '卡', 'n_strokes': '5'},
  {'zi': '角', 'n_strokes': '7'},
  {'zi': '责', 'n_strokes': '8'},
  {'zi': '乎', 'n_strokes': '5'},
  {'zi': '尸', 'n_strokes': '3'},
  {'zi': '釒', 'n_strokes': '8'},
  {'zi': '歹', 'n_strokes': '4'},
  {'zi': '五', 'n_strokes': '4'},
  {'zi': '出', 'n_strokes': '5'},
  {'zi': '谷', 'n_strokes': '7'},
  {'zi': '老', 'n_strokes': '6'},
  {'zi': '尤', 'n_strokes': '4'},
  {'zi': '豕', 'n_strokes': '7'},
  {'zi': '⺈', 'n_strokes': '2'},
  {'zi': '巨', 'n_strokes': '4'},
  {'zi': '血', 'n_strokes': '6'},
  {'zi': '户', 'n_strokes': '4'},
  {'zi': '麦', 'n_strokes': '7'},
  {'zi': '示', 'n_strokes': '5'},
  {'zi': '火', 'n_strokes': '4'},
  {'zi': '⺁', 'n_strokes': '2'},
  {'zi': '人', 'n_strokes': '2'},
  {'zi': '夫', 'n_strokes': '4'},
  {'zi': '巛', 'n_strokes': '3'},
  {'zi': '骨', 'n_strokes': '9'},
  {'zi': '气', 'n_strokes': '4'},
  {'zi': '寸', 'n_strokes': '3'},
  {'zi': '龺', 'n_strokes': '8'},
  {'zi': '士', 'n_strokes': '3'},
  {'zi': '呙', 'n_strokes': '7'},
  {'zi': '彑', 'n_strokes': '3'},
  {'zi': '式', 'n_strokes': '6'},
  {'zi': '专', 'n_strokes': '4'},
  {'zi': '月', 'n_strokes': '4'},
  {'zi': '玉', 'n_strokes': '5'},
  {'zi': '毛', 'n_strokes': '4'},
  {'zi': '⺲', 'n_strokes': '6'},
  {'zi': '用', 'n_strokes': '5'},
  {'zi': '井', 'n_strokes': '4'},
  {'zi': '罒', 'n_strokes': '5'},
  {'zi': '失', 'n_strokes': '5'},
  {'zi': '包', 'n_strokes': '5'},
  {'zi': '头', 'n_strokes': '5'},
  {'zi': '畏', 'n_strokes': '9'},
  {'zi': '匹', 'n_strokes': '4'},
  {'zi': '⺟', 'n_strokes': '4'},
  {'zi': '臣', 'n_strokes': '6'},
  {'zi': '元', 'n_strokes': '4'},
  {'zi': '曰', 'n_strokes': '4'},
  {'zi': '共', 'n_strokes': '6'},
  {'zi': '鳥', 'n_strokes': '11'},
  {'zi': '穴', 'n_strokes': '5'},
  {'zi': '甩', 'n_strokes': '5'},
  {'zi': '咸', 'n_strokes': '9'},
  {'zi': '合', 'n_strokes': '6'},
  {'zi': '亏', 'n_strokes': '3'},
  {'zi': '亢', 'n_strokes': '4'},
  {'zi': '丑', 'n_strokes': '4'},
  {'zi': '赱', 'n_strokes': '7'},
  {'zi': '占', 'n_strokes': '5'},
  {'zi': '本', 'n_strokes': '5'},
  {'zi': '攴', 'n_strokes': '4'},
  {'zi': '只', 'n_strokes': '5'},
  {'zi': '鼠', 'n_strokes': '13'},
  {'zi': '半', 'n_strokes': '5'},
  {'zi': '玄', 'n_strokes': '5'},
  {'zi': '罓', 'n_strokes': '4'},
  {'zi': '女', 'n_strokes': '3'},
  {'zi': '厂', 'n_strokes': '2'},
  {'zi': '佥', 'n_strokes': '7'},
  {'zi': '禸', 'n_strokes': '4'},
  {'zi': '丿', 'n_strokes': '1'},
  {'zi': '末', 'n_strokes': '5'},
  {'zi': '鼓', 'n_strokes': '13'},
  {'zi': '龴', 'n_strokes': '2'},
  {'zi': '斥', 'n_strokes': '5'},
  {'zi': '豆', 'n_strokes': '7'},
  {'zi': '升', 'n_strokes': '4'},
  {'zi': '天', 'n_strokes': '4'},
  {'zi': '去', 'n_strokes': '5'},
  {'zi': '上', 'n_strokes': '3'},
  {'zi': '内', 'n_strokes': '4'},
  {'zi': '食', 'n_strokes': '9'},
  {'zi': '隹', 'n_strokes': '8'},
  {'zi': '弓', 'n_strokes': '3'},
  {'zi': '正', 'n_strokes': '5'},
  {'zi': '身', 'n_strokes': '7'},
  {'zi': '乡', 'n_strokes': '3'},
  {'zi': '了', 'n_strokes': '2'},
  {'zi': '台', 'n_strokes': '5'},
  {'zi': '貝', 'n_strokes': '7'},
  {'zi': '目', 'n_strokes': '5'},
  {'zi': '旡', 'n_strokes': '4'},
  {'zi': '囗', 'n_strokes': '3'},
  {'zi': '齐', 'n_strokes': '6'},
  {'zi': '土', 'n_strokes': '3'},
  {'zi': '门', 'n_strokes': '3'},
  {'zi': '邑', 'n_strokes': '7'},
  {'zi': '戋', 'n_strokes': '5'},
  {'zi': '尼', 'n_strokes': '5'},
  {'zi': '力', 'n_strokes': '2'},
  {'zi': '开', 'n_strokes': '4'},
  {'zi': '叉', 'n_strokes': '3'},
  {'zi': '分', 'n_strokes': '4'},
  {'zi': '文', 'n_strokes': '4'},
  {'zi': '川', 'n_strokes': '3'},
  {'zi': '长', 'n_strokes': '4'},
  {'zi': '乛', 'n_strokes': '1'},
  {'zi': '钅', 'n_strokes': '5'},
  {'zi': '屮', 'n_strokes': '3'},
  {'zi': '幷', 'n_strokes': '8'},
  {'zi': '易', 'n_strokes': '8'},
  {'zi': '缶', 'n_strokes': '6'},
  {'zi': '羊', 'n_strokes': '6'},
  {'zi': '纟', 'n_strokes': '3'},
  {'zi': '艸', 'n_strokes': '6'},
  {'zi': '革', 'n_strokes': '9'},
  {'zi': '而', 'n_strokes': '6'},
  {'zi': '三', 'n_strokes': '3'},
  {'zi': '鹿', 'n_strokes': '11'},
  {'zi': '黄', 'n_strokes': '12'},
  {'zi': '介', 'n_strokes': '4'},
  {'zi': '朩', 'n_strokes': '4'},
  {'zi': '兀', 'n_strokes': '3'},
  {'zi': '手', 'n_strokes': '4'},
  {'zi': '兰', 'n_strokes': '5'},
  {'zi': '襾', 'n_strokes': '6'}]

###     attrib = term

In [33]:
data_term = [] 
n_max = 1000 # 10 # 
n = 0
DEBUG = False # True #  
for r in radicals:
    if r == ' ': continue

    sql_5 = f"""
        select '{r}' as zi, colloquial_term as term
        from sh_kangxi_radical_214 
        where trim(zi_radical) like '%{r}%'
    """
    df_5 = pd.read_sql(sql_5, conn)
    dd = df_5.to_dict("records")
    if dd: 
        if dd[0].get("term").strip(): 
            data_term += dd
        if DEBUG: 
            print(dd)
    
    n += 1 
    if n > n_max: break 
        

len(data_term), data_term

(91,
 [{'zi': '刀', 'term': '刀字旁 立刀旁'},
  {'zi': '車', 'term': '车字旁'},
  {'zi': '欠', 'term': '欠字旁'},
  {'zi': '⺍', 'term': '当字头'},
  {'zi': '⺮', 'term': '竹字头'},
  {'zi': '邑', 'term': '右耳朵'},
  {'zi': '竹', 'term': '竹字头'},
  {'zi': '几', 'term': '几字旁'},
  {'zi': '冖', 'term': '秃宝盖'},
  {'zi': '釒', 'term': '金字旁'},
  {'zi': '丬', 'term': '将字旁'},
  {'zi': '犬', 'term': '反犬旁'},
  {'zi': '⺗', 'term': '竖心旁'},
  {'zi': '冂', 'term': '同字匡'},
  {'zi': '彳', 'term': '双立人'},
  {'zi': '⺪', 'term': '疋字底'},
  {'zi': '厶', 'term': '私字旁'},
  {'zi': '火', 'term': '四点底 火字旁'},
  {'zi': '爫', 'term': '受字头'},
  {'zi': '⺧', 'term': '牛字旁'},
  {'zi': '⻌', 'term': '走之底'},
  {'zi': '龵', 'term': '提手旁 看字头'},
  {'zi': '糸', 'term': '绞丝旁'},
  {'zi': '廾', 'term': '弄字底'},
  {'zi': '廴', 'term': '建之旁'},
  {'zi': '老', 'term': '老字头'},
  {'zi': '⺈', 'term': '刀字旁 立刀旁'},
  {'zi': '艸', 'term': '草字头'},
  {'zi': '⺿', 'term': '草字头'},
  {'zi': '食', 'term': '食字旁'},
  {'zi': '弓', 'term': '弓字旁'},
  {'zi': '疋', 'term': '疋字底'},
  {'zi': '亠', 'term

In [34]:
data_term = [
    {'zi': '彳', 'term': '双立人'},
  {'zi': '疋', 'term': '疋字底'},
  {'zi': '辵', 'term': '走之底'},
  {'zi': '亠', 'term': '六字头'},
  {'zi': '禾', 'term': '禾木旁'},
  {'zi': '入', 'term': '入字旁'},
  {'zi': '亻', 'term': '单立人 单人旁'},
  {'zi': '竹', 'term': '竹字头'},
  {'zi': '疒', 'term': '病字旁'},
  {'zi': '牜', 'term': '牛字旁'},
  {'zi': '厶', 'term': '私字旁'},
  {'zi': '欠', 'term': '欠字旁'},
  {'zi': '勹', 'term': '包字头'},
  {'zi': '耂', 'term': '老字头'},
  {'zi': '癶', 'term': '登字头'},
  {'zi': '⺪', 'term': '疋字底'},
  {'zi': '巾', 'term': '巾字儿'},
  {'zi': '小', 'term': '当字头'},
  {'zi': '犭', 'term': '反犬旁'},
  {'zi': '氺', 'term': '三点水'},
  {'zi': '糸', 'term': '绞丝旁'},
  {'zi': '金', 'term': '金字旁'},
  {'zi': '木', 'term': '木字旁'},
  {'zi': '犬', 'term': '反犬旁'},
  {'zi': '爫', 'term': '受字头'},
  {'zi': '⺧', 'term': '牛字旁'},
  {'zi': '米', 'term': '米字旁'},
  {'zi': '⺿', 'term': '草字头'},
  {'zi': '扌', 'term': '提手旁'},
  {'zi': '訁', 'term': '言字旁'},
  {'zi': '車', 'term': '车字旁'},
  {'zi': '匚', 'term': '医字框'},
  {'zi': '卩', 'term': '单耳刀'},
  {'zi': '子', 'term': '子字旁'},
  {'zi': '⻌', 'term': '走之底'},
  {'zi': '糹', 'term': '绞丝旁'},
  {'zi': '⺍', 'term': '当字头'},
  {'zi': '㔾', 'term': '单耳刀'},
  {'zi': '氵', 'term': '三点水'},
  {'zi': '凵', 'term': '凶字框'},
  {'zi': '刂', 'term': '刀字旁 立刀旁'},
  {'zi': '丶', 'term': '丶字旁'},
  {'zi': '几', 'term': '几字旁'},
  {'zi': '牛', 'term': '牛字旁'},
  {'zi': '冫', 'term': '两点水'},
  {'zi': '⺌', 'term': '当字头'},
  {'zi': '爿', 'term': '将字旁'},
  {'zi': '足', 'term': '足字旁'},
  {'zi': '⻊', 'term': '足字旁'},
  {'zi': '忄', 'term': '竖心旁'},
  {'zi': '广', 'term': '广字旁'},
  {'zi': '灬', 'term': '四点底 火字旁'},
  {'zi': '廴', 'term': '建之旁'},
  {'zi': '衣', 'term': '衣字旁'},
  {'zi': '⺫', 'term': '目字旁'},
  {'zi': '一', 'term': '一字旁'},
  {'zi': '宀', 'term': '宝盖头'},
  {'zi': '⻂', 'term': '衣字旁'},
  {'zi': '⺮', 'term': '竹字头'},
  {'zi': '彡', 'term': '三撇儿'},
  {'zi': '冂', 'term': '同字匡'},
  {'zi': '飠', 'term': '食字旁'},
  {'zi': '⻏', 'term': '右耳朵'},
  {'zi': '龵', 'term': '看字头'},
  {'zi': '門', 'term': '门字框'},
  {'zi': '爪', 'term': '受字头'},
  {'zi': '廾', 'term': '弄字底'},
  {'zi': '刀', 'term': '刀字旁 立刀旁'},
  {'zi': '⻎', 'term': '走之底'},
  {'zi': '⺗', 'term': '竖心旁'},
  {'zi': '冖', 'term': '秃宝盖'},
  {'zi': '丬', 'term': '将字旁'},
  {'zi': '礻', 'term': '示字旁'},
  {'zi': '水', 'term': '三点水'},
  {'zi': '言', 'term': '言字旁'},
  {'zi': '心', 'term': '竖心旁'},
  {'zi': '⻍', 'term': '走之底'},
  {'zi': '釒', 'term': '金字旁'},
  {'zi': '老', 'term': '老字头'},
  {'zi': '⺈', 'term': '刀字旁 立刀旁'},
  {'zi': '示', 'term': '示字旁'},
  {'zi': '火', 'term': '火字旁'},
  {'zi': '人', 'term': '单立人 单人旁'},
  {'zi': '食', 'term': '食字旁'},
  {'zi': '弓', 'term': '弓字旁'},
  {'zi': '目', 'term': '目字旁'},
  {'zi': '囗', 'term': '国字框'},
  {'zi': '土', 'term': '提土旁'},
  {'zi': '邑', 'term': '右耳朵'},
  {'zi': '艸', 'term': '草字头'},
  {'zi': '手', 'term': '提手旁'}]

###     attrib = examples

In [35]:
data_examples = [] 
n_max = 1000 # 10 # 
n = 0
DEBUG = False # True #  
for r in radicals:
    if r == ' ': continue

    sql_5 = f"""
        select '{r}' as zi, examples
        from sh_kangxi_radical_214 
        where trim(zi_radical) like '%{r}%'
    """
    df_5 = pd.read_sql(sql_5, conn)
    dd = df_5.to_dict("records")
    if dd: 
        if dd[0].get("examples"): 
            data_examples += dd
        if DEBUG: 
            print(dd)
    
    n += 1 
    if n > n_max: break 
        

len(data_examples), data_examples

(282,
 [{'zi': '豕', 'examples': '豗、豚、豜'},
  {'zi': '夊', 'examples': '夌、复、夏'},
  {'zi': '⺷', 'examples': '羋、羌、美'},
  {'zi': '刀', 'examples': '刁、解、分、劍'},
  {'zi': '車', 'examples': '軋、軌、軍'},
  {'zi': '風', 'examples': '颩、颭、颮'},
  {'zi': '支', 'examples': '攰、攱、攲'},
  {'zi': '鬥', 'examples': '鬦、鬧、鬨'},
  {'zi': '斗', 'examples': '料、斚、斛'},
  {'zi': '黑', 'examples': '墨、黓、黔'},
  {'zi': '欠', 'examples': '次、欣、欥'},
  {'zi': '⺍', 'examples': '少、尖、當、栄'},
  {'zi': '⺮', 'examples': '竺、竽、竿'},
  {'zi': '襾', 'examples': '西、要、覂'},
  {'zi': '鼻', 'examples': '鼽、鼾、齁'},
  {'zi': '龠', 'examples': '龡、龢、龤'},
  {'zi': '⺲', 'examples': '罔、罕、罘'},
  {'zi': '方', 'examples': '㫃、於、㫄'},
  {'zi': '邑', 'examples': '邕、邗、邘'},
  {'zi': '彐', 'examples': '彔、彖、彗'},
  {'zi': '肉', 'examples': '肊、肋、然'},
  {'zi': '竹', 'examples': '竺、竽、竿'},
  {'zi': '香', 'examples': '馝、馞、馡'},
  {'zi': '几', 'examples': '風、殳、凡、凱'},
  {'zi': '川', 'examples': '川、州、順、災'},
  {'zi': '辛', 'examples': '辜、辝、辟'},
  {'zi': '冖', 'examples': '冗、冠、冥、運'},
  {'zi': '釒'

In [36]:
data_examples = [{'zi': '么', 'examples': '幻、麼、幽、幾'},
  {'zi': '⺷', 'examples': '羋、羌、美'},
  {'zi': '戶', 'examples': '戹、戼、戽'},
  {'zi': '耒', 'examples': '耔、耕、耖'},
  {'zi': '彳', 'examples': '㣔、彴、彷'},
  {'zi': '龠', 'examples': '龡、龢、龤'},
  {'zi': '魚', 'examples': '魟、魠、魦'},
  {'zi': '疋', 'examples': '疌、疏、疎'},
  {'zi': '攵', 'examples': '收、攷、攸'},
  {'zi': '舟', 'examples': '舠、舡、舢'},
  {'zi': '虍', 'examples': '虎、虐、虒'},
  {'zi': '辵', 'examples': '边、巡、迂'},
  {'zi': '阜', 'examples': '阞、阡、阢'},
  {'zi': '亠', 'examples': '方、亡、亢、交'},
  {'zi': '麻', 'examples': '麼、麾、黁'},
  {'zi': '赤', 'examples': '赦、赧、赨'},
  {'zi': '⺻', 'examples': '肄、肆、肅'},
  {'zi': '青', 'examples': '靖、静、靚'},
  {'zi': '耳', 'examples': '耴、耵、耷'},
  {'zi': '見', 'examples': '規、覓、視'},
  {'zi': '禾', 'examples': '禿、秀、私'},
  {'zi': '己', 'examples': '記、改、忌、龍'},
  {'zi': '丨', 'examples': '十、中、串、丰'},
  {'zi': '辰', 'examples': '辱、農、辴'},
  {'zi': '入', 'examples': '內、全、兩、汆'},
  {'zi': '石', 'examples': '矸、矻、矼'},
  {'zi': '儿', 'examples': '兒、兀、允、元'},
  {'zi': '皮', 'examples': '皯、皰、皴'},
  {'zi': '韋', 'examples': '韌、韍、韎'},
  {'zi': '匸', 'examples': '亡、匹、匼、匽'},
  {'zi': '雨', 'examples': '雩、雪、雯'},
  {'zi': '十', 'examples': '千、半、卅、升'},
  {'zi': '亻', 'examples': '你、什、仁、仇'},
  {'zi': '竹', 'examples': '竺、竽、竿'},
  {'zi': '疒', 'examples': '疔、疚、疝'},
  {'zi': '黽', 'examples': '黿、鼀、鼁'},
  {'zi': '牜', 'examples': '牝、牟、牠'},
  {'zi': '厶', 'examples': '台、公、去、參、'},
  {'zi': '欠', 'examples': '次、欣、欥'},
  {'zi': '頁', 'examples': '頂、頃、頄'},
  {'zi': '勹', 'examples': '包、勺、勻、勾'},
  {'zi': '虫', 'examples': '虬、虯、虱'},
  {'zi': '高', 'examples': '髚、髛、𩫛'},
  {'zi': '干', 'examples': '旱、开、平、年'},
  {'zi': '耂', 'examples': '考、者、耆'},
  {'zi': '癶', 'examples': '癸、癹、発'},
  {'zi': '斤', 'examples': '斥、斧、斨'},
  {'zi': '鼎', 'examples': '鼏、鼐、鼒'},
  {'zi': '⺪', 'examples': '疌、疏、疎'},
  {'zi': '比', 'examples': '毖、毗、毘'},
  {'zi': '羽', 'examples': '羿、翀、翁'},
  {'zi': '片', 'examples': '版、牉、牌'},
  {'zi': '自', 'examples': '臬、臭、臯'},
  {'zi': '矢', 'examples': '矣、知、矧'},
  {'zi': '又', 'examples': '叉、及、友、取'},
  {'zi': '行', 'examples': '衍、衎、衒'},
  {'zi': '里', 'examples': '重、野、量'},
  {'zi': '巾', 'examples': '市、帥、刷、砸'},
  {'zi': '肉', 'examples': '肊、肋、然'},
  {'zi': '丷', 'examples': '公、六、兮、穴'},
  {'zi': '小', 'examples': '少、尖、當、栄'},
  {'zi': '口', 'examples': '哎、古、只、品'},
  {'zi': '夊', 'examples': '夌、复、夏'},
  {'zi': '鬯', 'examples': '鬰、鬱'},
  {'zi': '犭', 'examples': '犮、犯、犰'},
  {'zi': '斉', 'examples': '齋、齌、齍'},
  {'zi': '氺', 'examples': '永、氷、氾'},
  {'zi': '生', 'examples': '甡、產、甥'},
  {'zi': '糸', 'examples': '系、糾、紀'},
  {'zi': '金', 'examples': '釓、釔、釕'},
  {'zi': '首', 'examples': '馗、䭫、馘'},
  {'zi': '木', 'examples': '未、末、本'},
  {'zi': '西', 'examples': '西、要、覂'},
  {'zi': '麥', 'examples': '麨、麩、麪'},
  {'zi': '犬', 'examples': '犮、犯、犰'},
  {'zi': '飛', 'examples': '䬡、飜、飝'},
  {'zi': '爫', 'examples': '爬、爭、爰'},
  {'zi': '⺺', 'examples': '肄、肆、肅'},
  {'zi': '⺧', 'examples': '牝、牟、牠'},
  {'zi': '色', 'examples': '艳、艴、艵'},
  {'zi': '矛', 'examples': '矜、矞、矟'},
  {'zi': '米', 'examples': '籸、籹、籽'},
  {'zi': '舌', 'examples': '舍、舐、舑'},
  {'zi': '鹵', 'examples': '鹹、鹺、鹼'},
  {'zi': '酉', 'examples': '酊、酋、酌'},
  {'zi': '⺿', 'examples': '艽、艾、芃'},
  {'zi': '鬥', 'examples': '鬦、鬧、鬨'},
  {'zi': '牙', 'examples': '㸦、㸧、牚'},
  {'zi': '母', 'examples': '母、每、毐'},
  {'zi': '扌', 'examples': '才、扎、扐'},
  {'zi': '匕', 'examples': '化、北、匙、比'},
  {'zi': '斗', 'examples': '料、斚、斛'},
  {'zi': '訁', 'examples': '訂、訃、計'},
  {'zi': '鬼', 'examples': '鬾、魁、魂'},
  {'zi': '車', 'examples': '軋、軌、軍'},
  {'zi': '⻖', 'examples': '阞、阡、阢'},
  {'zi': '風', 'examples': '颩、颭、颮'},
  {'zi': '歺', 'examples': '歺、死、歿'},
  {'zi': '网', 'examples': '罔、罕、罘'},
  {'zi': '尢', 'examples': '尤、尷、尬、尩'},
  {'zi': '匚', 'examples': '匿、區、巨、匠'},
  {'zi': '卩', 'examples': '即、卬、卷、夗'},
  {'zi': '子', 'examples': '孩、學、孔、仔'},
  {'zi': '辛', 'examples': '辜、辝、辟'},
  {'zi': '⻌', 'examples': '边、巡、迂'},
  {'zi': '无', 'examples': '既、旣、旤'},
  {'zi': '黑', 'examples': '墨、黓、黔'},
  {'zi': '龍', 'examples': '龏、龑、龔'},
  {'zi': '田', 'examples': '由、甲、申'},
  {'zi': '音', 'examples': '竟、章、韵'},
  {'zi': '長', 'examples': '镺、镻、镼'},
  {'zi': '糹', 'examples': '系、糾、紀'},
  {'zi': '⺍', 'examples': '少、尖、當、栄'},
  {'zi': '⺳', 'examples': '罔、罕、罘'},
  {'zi': '臼', 'examples': '臾、臿、舁'},
  {'zi': '乚', 'examples': '九、乞、也'},
  {'zi': '㔾', 'examples': '即、卬、卷、夗'},
  {'zi': '尣', 'examples': '尤、尷、尬、尩'},
  {'zi': '香', 'examples': '馝、馞、馡'},
  {'zi': '殳', 'examples': '段、殷、殺'},
  {'zi': '戈', 'examples': '戊、戉、戌'},
  {'zi': '氵', 'examples': '永、氷、氾'},
  {'zi': '鬲', 'examples': '鬳、鬴、鬵'},
  {'zi': '釆', 'examples': '采、釉、釋'},
  {'zi': '凵', 'examples': '凶、凸、凹、齒'},
  {'zi': '走', 'examples': '赳、赴、赶'},
  {'zi': '刂', 'examples': '刁、解、分、劍'},
  {'zi': '丶', 'examples': '丸、凡、丹、户'},
  {'zi': '隶', 'examples': '𨽻、隷、隸'},
  {'zi': '卜', 'examples': '下、上、卡、占'},
  {'zi': '方', 'examples': '㫃、於、㫄'},
  {'zi': '瓜', 'examples': '瓝、瓞、瓟'},
  {'zi': '彐', 'examples': '彔、彖、彗'},
  {'zi': '二', 'examples': '貳、于、云、些'},
  {'zi': '非', 'examples': '靟、靠、靡'},
  {'zi': '几', 'examples': '風、殳、凡、凱'},
  {'zi': '皿', 'examples': '盂、盃、盅'},
  {'zi': '王', 'examples': '玻、瑪、璧'},
  {'zi': '牛', 'examples': '牝、牟、牠'},
  {'zi': '冫', 'examples': '冬、冰、冶、凉'},
  {'zi': '⺼', 'examples': '肊、肋、然'},
  {'zi': '⺌', 'examples': '少、尖、當、栄'},
  {'zi': '爿', 'examples': '牀、牁、牂'},
  {'zi': '白', 'examples': '百、皀、皁'},
  {'zi': '足', 'examples': '趴、趵、趷'},
  {'zi': '⻊', 'examples': '趴、趵、趷'},
  {'zi': '忄', 'examples': '必、忉、忌'},
  {'zi': '乙', 'examples': '九、乞、也'},
  {'zi': '鼻', 'examples': '鼽、鼾、齁'},
  {'zi': '氏', 'examples': '氐、民、氓'},
  {'zi': '广', 'examples': '庀、庂、庄'},
  {'zi': '灬', 'examples': '灰、灯、灶'},
  {'zi': '日', 'examples': '旦、旨、早'},
  {'zi': '龜', 'examples': '䶰、龝、龞'},
  {'zi': '八', 'examples': '公、六、兮、穴'},
  {'zi': '黹', 'examples': '黺、黻、黼'},
  {'zi': '廴', 'examples': '廵、廷、延'},
  {'zi': '立', 'examples': '竑、竒、竘'},
  {'zi': '衣', 'examples': '初、表、衫'},
  {'zi': '黃', 'examples': '黈、䵍、黌'},
  {'zi': '工', 'examples': '左、巧、功、式'},
  {'zi': '⺫', 'examples': '盯、盱、盲'},
  {'zi': '大', 'examples': '太、天、夭、夯'},
  {'zi': '一', 'examples': '王、丁、七、三'},
  {'zi': '⺶', 'examples': '羋、羌、美'},
  {'zi': '髙', 'examples': '髚、髛、𩫛'},
  {'zi': '宀', 'examples': '家、安、寧、它'},
  {'zi': '⻂', 'examples': '初、表、衫'},
  {'zi': '乀', 'examples': '乂、乃、久、八'},
  {'zi': '止', 'examples': '正、此、步'},
  {'zi': '⺮', 'examples': '竺、竽、竿'},
  {'zi': '彡', 'examples': '形、彤、彥'},
  {'zi': '黍', 'examples': '黎、黏、黐'},
  {'zi': '冂', 'examples': '冇、冊、冉、肉'},
  {'zi': '齒', 'examples': '齔、齕、齖'},
  {'zi': '飠', 'examples': '飡、飢、飣'},
  {'zi': '⻏', 'examples': '邕、邗、邘'},
  {'zi': '支', 'examples': '攰、攱、攲'},
  {'zi': '玊', 'examples': '玻、瑪、璧'},
  {'zi': '龵', 'examples': '才、扎、扐'},
  {'zi': '夂', 'examples': '各、冬、夆、愛'},
  {'zi': '瓦', 'examples': '㼚、瓮、瓫'},
  {'zi': '幺', 'examples': '幻、麼、幽、幾'},
  {'zi': '戸', 'examples': '戹、戼、戽'},
  {'zi': '毋', 'examples': '母、每、毐'},
  {'zi': '甘', 'examples': '甚、甜、甛'},
  {'zi': '亅', 'examples': '了、矛、事'},
  {'zi': '靑', 'examples': '靖、静、靚'},
  {'zi': '舛', 'examples': '舜、舝、舞'},
  {'zi': '至', 'examples': '致、臷、臺'},
  {'zi': '髟', 'examples': '髠、髡、髢'},
  {'zi': '門', 'examples': '閂、閃、閆'},
  {'zi': '山', 'examples': '嵗、密、峰、幽'},
  {'zi': '面', 'examples': '靤、靦、靧'},
  {'zi': '爪', 'examples': '爬、爭、爰'},
  {'zi': '聿', 'examples': '肄、肆、肅'},
  {'zi': '豸', 'examples': '豺、豻、豹'},
  {'zi': '廾', 'examples': '廿、弁、弄'},
  {'zi': '父', 'examples': '爸、爹、爺'},
  {'zi': '镸', 'examples': '镺、镻、镼'},
  {'zi': '刀', 'examples': '刁、解、分、劍'},
  {'zi': '⻎', 'examples': '边、巡、迂'},
  {'zi': '⺗', 'examples': '必、忉、忌'},
  {'zi': '夕', 'examples': '外、夗、多、岁'},
  {'zi': '冖', 'examples': '冗、冠、冥、運'},
  {'zi': '丬', 'examples': '牀、牁、牂'},
  {'zi': '⺄', 'examples': '乂、乃、久、八'},
  {'zi': '靣', 'examples': '靤、靦、靧'},
  {'zi': '⻇', 'examples': '觓、觔、觕'},
  {'zi': '礻', 'examples': '礼、礽、社'},
  {'zi': '水', 'examples': '永、氷、氾'},
  {'zi': '馬', 'examples': '馭、馮、馯'},
  {'zi': '言', 'examples': '訂、訃、計'},
  {'zi': '心', 'examples': '必、忉、忌'},
  {'zi': '爻', 'examples': '爼、爽、爾'},
  {'zi': '覀', 'examples': '西、要、覂'},
  {'zi': '韭', 'examples': '韰、韲、䪢'},
  {'zi': '齊', 'examples': '齋、齌、齍'},
  {'zi': '艮', 'examples': '良、艱'},
  {'zi': '⻍', 'examples': '边、巡、迂'},
  {'zi': '弋', 'examples': '弌、弍、弎'},
  {'zi': '角', 'examples': '觓、觔、觕'},
  {'zi': '尸', 'examples': '屍、尺、尼、尻'},
  {'zi': '釒', 'examples': '釓、釔、釕'},
  {'zi': '歹', 'examples': '歺、死、歿'},
  {'zi': '谷', 'examples': '谹、谽、谿'},
  {'zi': '老', 'examples': '考、者、耆'},
  {'zi': '豕', 'examples': '豗、豚、豜'},
  {'zi': '⺈', 'examples': '刁、解、分、劍'},
  {'zi': '血', 'examples': '衁、衂、衃'},
  {'zi': '户', 'examples': '戹、戼、戽'},
  {'zi': '示', 'examples': '礼、礽、社'},
  {'zi': '火', 'examples': '灰、灯、灶'},
  {'zi': '人', 'examples': '你、什、仁、仇'},
  {'zi': '巛', 'examples': '川、州、順、災'},
  {'zi': '骨', 'examples': '骫、骭、骯'},
  {'zi': '气', 'examples': '氕、氘、氚'},
  {'zi': '寸', 'examples': '村、寺、封、射'},
  {'zi': '士', 'examples': '壬、喜、時、壽'},
  {'zi': '彑', 'examples': '彔、彖、彗'},
  {'zi': '月', 'examples': '有、朋、服'},
  {'zi': '玉', 'examples': '玻、瑪、璧'},
  {'zi': '毛', 'examples': '毡、毣、毧'},
  {'zi': '⺲', 'examples': '罔、罕、罘'},
  {'zi': '用', 'examples': '甩、甫、甬'},
  {'zi': '臣', 'examples': '臤、臥、臧'},
  {'zi': '曰', 'examples': '曲、曳、更'},
  {'zi': '鳥', 'examples': '鳦、鳧、鳩'},
  {'zi': '穴', 'examples': '究、穸、空'},
  {'zi': '攴', 'examples': '收、攷、攸'},
  {'zi': '鼠', 'examples': '鼢、鼥、鼩'},
  {'zi': '玄', 'examples': '玅、率、玈'},
  {'zi': '罓', 'examples': '罔、罕、罘'},
  {'zi': '女', 'examples': '妳、婆、奶、如'},
  {'zi': '厂', 'examples': '厄、原、厚、厰'},
  {'zi': '禸', 'examples': '禹、禺、离'},
  {'zi': '丿', 'examples': '乂、乃、久、八'},
  {'zi': '鼓', 'examples': '鼕、鼖、鼗'},
  {'zi': '豆', 'examples': '豇、豈、豉'},
  {'zi': '食', 'examples': '飡、飢、飣'},
  {'zi': '隹', 'examples': '隻、隼、隽'},
  {'zi': '弓', 'examples': '弔、引、弗'},
  {'zi': '身', 'examples': '躬、躭、躰'},
  {'zi': '貝', 'examples': '貞、負、財'},
  {'zi': '目', 'examples': '盯、盱、盲'},
  {'zi': '旡', 'examples': '既、旣、旤'},
  {'zi': '囗', 'examples': '囚、四、國、圍'},
  {'zi': '土', 'examples': '地、圣、圭、堯'},
  {'zi': '邑', 'examples': '邕、邗、邘'},
  {'zi': '力', 'examples': '加、功、劣、男'},
  {'zi': '文', 'examples': '斉、斌、斐'},
  {'zi': '川', 'examples': '川、州、順、災'},
  {'zi': '乛', 'examples': '九、乞、也'},
  {'zi': '屮', 'examples': '艸、屯、屰、頓'},
  {'zi': '缶', 'examples': '缸、缺、缽'},
  {'zi': '羊', 'examples': '羋、羌、美'},
  {'zi': '艸', 'examples': '艽、艾、芃'},
  {'zi': '革', 'examples': '靭、靮、靳'},
  {'zi': '而', 'examples': '耍、耎、耏'},
  {'zi': '鹿', 'examples': '麀、麁、麂'},
  {'zi': '手', 'examples': '才、扎、扐'},
  {'zi': '襾', 'examples': '西、要、覂'}]

In [37]:
def merge_zi_dataframes(df1, df2):
    """
    Merge two dataframes on 'zi' column using left merge
    
    Parameters:
    df1: First DataFrame with columns ['zi', 'is_zi']
    df2: Second DataFrame with columns ['zi', 'pinyin']
    
    Returns:
    Merged DataFrame with columns ['zi', 'is_zi', 'pinyin']
    """
    # Perform left merge on 'zi' column
    merged_df = pd.merge(
        left=df1,
        right=df2,
        on='zi',
        how='left'
    ).fillna("")
    
    return merged_df

## Create dataframes and merge

In [38]:
df_ = create_df_from_list(data_is_zi)

In [39]:
df_.head(10)

,zi,is_zi
0,么,1
1,以,1
2,⺷,0
3,丈,1
4,戶,1
5,耒,1
6,彳,0
7,东,1
8,疋,0
9,攵,0


In [40]:
df_id_kangxi = create_df_from_list(data_id_kangxi) 
df_ = merge_zi_dataframes(df_, df_id_kangxi)
df_.head(10)

,zi,is_zi,id_kangxi
0,么,1,
1,以,1,
2,⺷,0,
3,丈,1,
4,戶,1,
5,耒,1,127.0
6,彳,0,60.0
7,东,1,
8,疋,0,
9,攵,0,


In [41]:
df_meaning = create_df_from_list(data_meaning) 
df_ = merge_zi_dataframes(df_, df_meaning)
df_.head(10)

,zi,is_zi,id_kangxi,meaning
0,么,1,,short thread
1,以,1,,
2,⺷,0,,sheep
3,丈,1,,meter
4,戶,1,,door
5,耒,1,127.0,plow / plough
6,彳,0,60.0,walk / step
7,东,1,,
8,疋,0,,foot / bolt of cloth
9,攵,0,,tap / rap


In [42]:
df_pinyin = create_df_from_list(data_pinyin)
df_ = merge_zi_dataframes(df_, df_pinyin)
df_.head(10)

,zi,is_zi,id_kangxi,meaning,pinyin
0,么,1,,short thread,yāo
1,以,1,,,
2,⺷,0,,sheep,yáng
3,丈,1,,meter,
4,戶,1,,door,hù
5,耒,1,127.0,plow / plough,li / lěi
6,彳,0,60.0,walk / step,chén / chì
7,东,1,,,
8,疋,0,,foot / bolt of cloth,pǐ / shū
9,攵,0,,tap / rap,pū


In [43]:
df_n_strokes = create_df_from_list(data_n_strokes)
df_ = merge_zi_dataframes(df_, df_n_strokes)
df_.head(10)

,zi,is_zi,id_kangxi,meaning,pinyin,n_strokes
0,么,1,,short thread,yāo,3
1,以,1,,,,4
2,⺷,0,,sheep,yáng,6
3,丈,1,,meter,,3
4,戶,1,,door,hù,4
5,耒,1,127.0,plow / plough,li / lěi,6
6,彳,0,60.0,walk / step,chén / chì,3
7,东,1,,,,5
8,疋,0,,foot / bolt of cloth,pǐ / shū,5
9,攵,0,,tap / rap,pū,4


In [44]:
df_term = create_df_from_list(data_term)
df_ = merge_zi_dataframes(df_, df_term)
df_.head(10)

,zi,is_zi,id_kangxi,meaning,pinyin,n_strokes,term
0,么,1,,short thread,yāo,3,
1,以,1,,,,4,
2,⺷,0,,sheep,yáng,6,
3,丈,1,,meter,,3,
4,戶,1,,door,hù,4,
5,耒,1,127.0,plow / plough,li / lěi,6,
6,彳,0,60.0,walk / step,chén / chì,3,双立人
7,东,1,,,,5,
8,疋,0,,foot / bolt of cloth,pǐ / shū,5,疋字底
9,攵,0,,tap / rap,pū,4,


In [45]:
df_examples = create_df_from_list(data_examples)
df_ = merge_zi_dataframes(df_, df_examples)
df_.head(10)

,zi,is_zi,id_kangxi,meaning,pinyin,n_strokes,term,examples
0,么,1,,short thread,yāo,3,,幻、麼、幽、幾
1,以,1,,,,4,,
2,⺷,0,,sheep,yáng,6,,羋、羌、美
3,丈,1,,meter,,3,,
4,戶,1,,door,hù,4,,戹、戼、戽
5,耒,1,127.0,plow / plough,li / lěi,6,,耔、耕、耖
6,彳,0,60.0,walk / step,chén / chì,3,双立人,㣔、彴、彷
7,东,1,,,,5,,
8,疋,0,,foot / bolt of cloth,pǐ / shū,5,疋字底,疌、疏、疎
9,攵,0,,tap / rap,pū,4,,收、攷、攸


In [46]:
df_variant = create_df_from_list(data_variant)
df_ = merge_zi_dataframes(df_, df_variant)
df_.head(10)

,zi,is_zi,id_kangxi,meaning,pinyin,n_strokes,term,examples,variant
0,么,1,,short thread,yāo,3,,幻、麼、幽、幾,
1,以,1,,,,4,,,
2,⺷,0,,sheep,yáng,6,,羋、羌、美,
3,丈,1,,meter,,3,,,
4,戶,1,,door,hù,4,,戹、戼、戽,
5,耒,1,127.0,plow / plough,li / lěi,6,,耔、耕、耖,
6,彳,0,60.0,walk / step,chén / chì,3,双立人,㣔、彴、彷,
7,东,1,,,,5,,,
8,疋,0,,foot / bolt of cloth,pǐ / shū,5,疋字底,疌、疏、疎,
9,攵,0,,tap / rap,pū,4,,收、攷、攸,


In [47]:
df_.shape 

(419, 9)

In [48]:
df_.to_csv("elemental_zi_v3.csv", index=False)